In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import io
import os
import sys
from typing import (
    Dict, 
    List, 
    Tuple,
    Any
)
import warnings

from tqdm import tqdm
from datetime import datetime

import numpy as np
import pandas as pd
from joblib import load, dump

from optbinning import OptimalBinning

import seaborn as sns
import matplotlib.pyplot as plt

(CVXPY) Jun 11 06:24:07 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jun 11 06:24:07 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [3]:
sys.path.append(os.path.join(os.getcwd(), '..'))
from utils.metrics import Metrics

In [4]:
# set constants
COLUMNS_DICT: Dict[str, str] = {'OPTIN_11SCN_ML': 'Num', 
'G5_ND_SPECII': 'Num', 
'SP_F2_33': 'Num', 
'AFF_B_09_01': 'Num', 
'TRD_A_20': 'Num', 
'SP_I_62_01': 'Num', 
'TRD_A_08': 'Num', 
'VM01_SP_VM2_15': 'Num', 
# 'GEN11_SP_N_92': 'Char', 
'VM07_SP_VM1_11': 'Num', 
'ND_ECC_08': 'Num', 
'CLU_NPR_L1M': 'Num', 
'TRD_P_12': 'Num', 
'SP_H_41': 'Num', 
'G5_ND_SPACII': 'Num', 
'TRD_B_24': 'Num', 
'SP_I_61_01': 'Num', 
'TRD_O_16': 'Num', 
'SP_B2_21': 'Num', 
'TRD_P_11': 'Num', 
'VM01_SP_VM2_23': 'Num', 
'SP_G_37': 'Num', 
'E4_Q_17': 'Char', 
'TRD_C_07': 'Num', 
'TRD_C_03': 'Num', 
'TRD_B_20': 'Num', 
'SP_B2_18': 'Num', 
'TRD_B_22': 'Num', 
'SP_B3_23': 'Num', 
'TRD_P_10': 'Num', 
'SPA_F2_33': 'Num', 
'VM01_SP_VM2_14': 'Num', 
'TRD_B_19': 'Num', 
'VM01_SP_VM2_24': 'Num', 
'SP_G_38': 'Num', 
'TRD_O_05': 'Num', 
'SPA_B2_18': 'Num', 
'E1_A_11': 'Num', 
'E1_A_09': 'Num', 
'E1_A_10': 'Num', 
'PTSBR_L6M_NPR_L6M': 'Num', 
'E2_H_06': 'Num', 
'SP_F1_30': 'Num', 
'TRD_O_01': 'Num', 
'TRD_O_12': 'Num', 
'TRD_P_09': 'Num', 
'TRD_O_07': 'Num', 
'E1_A_05': 'Num', 
'ND_INC_03': 'Num', 
'VM01_SP_VM2_17': 'Num', 
'SP_B1_14': 'Num', 
'VM01_SP_VM2_18': 'Num', 
'GEN11_SP_N_91': 'Num', 
'GEN11_SP_EDI_07': 'Num', 
'AFF_T_66': 'Char', 
'PTBR_L3M_NPR_L3M': 'Num', 
'SP_EDI_03': 'Num', 
'ND_ECC_06': 'Num', 
'TRD_B_18': 'Num', 
'SPA_B2_19': 'Num', 
'SP_E1_28': 'Num', 
'SP_I_64_01': 'Num', 
'VM07_SP_VM1_18': 'Num', 
'NUM_DELINQ_ACCS': 'Num', 
'TRD_C_09': 'Num', 
'SP_F1_31': 'Num', 
'TRD_B_17': 'Num', 
'TRD_C_10': 'Num', 
'TRD_B_08': 'Num', 
'EA5_S_01': 'Num', 
'ND_HAC_04': 'Num', 
'VM01_SP_VM2_26': 'Num', 
'SPA_A_04': 'Num', 
'CLU_CLI_L6M_NPR_L6M': 'Num', 
'E1_B_06': 'Num', 
'TRD_A_04': 'Num', 
# 'GEN11_SP_K_80': 'Char', 
'TRD_B_07': 'Num', 
'TRD_A_13': 'Num', 
'SP_I_63_01': 'Num', 
'SP_B2_20': 'Num', 
'E4_Q_04': 'Num', 
'TRD_RAG_01': 'Num', 
'TRD_A_03': 'Num', 
'VM09_SP_VM1_19': 'Num', 
'ND_ECC_04': 'Num', 
'VM08_SP_VM2_15': 'Num', 
'SP_B1_13': 'Num', 
'VAL_DELINQ_ACCS': 'Num', 
'AGE_MOST_RECENT_DELINQ': 'Num', 
'E1_A_03': 'Num', 
'E1_A_02': 'Num', 
# 'ND_ERL_01': 'Num', 
'VM08_SP_VM2_20': 'Num', 
'EA1_D_02': 'Num', 
'SP_I_56_01': 'Num', 
'AFF_E_07_01': 'Num', 
'TRD_B_02': 'Num', 
'SP_EDI_17': 'Char', 
'TRD_B_36': 'Num', 
'TRD_O_06': 'Num', 
'SP_EDI_26': 'Num', 
'VM02_SP_VM1_16': 'Num', 
'TRD_B_05': 'Num', 
'E2_G_08': 'Num', 
'AFF_T_64_01': 'Char', 
'SP_EDI_15': 'Char', 
'TRD_B_32': 'Num', 
'SP_EDI_27': 'Num', 
'NUM_SETTLED_ACCS': 'Num', 
'TRD_B_06': 'Num', 
'VM08_SP_VM2_16': 'Num', 
'SP_EDI_37': 'Num', 
'VM04_SP_VM1_19': 'Num', 
'E2_H_04': 'Num', 
'TRD_B_52': 'Num', 
'SP_B1_11': 'Num', 
'AGE_MOST_RECENT_SETTD': 'Num', 
'AFF_E_29': 'Char', 
'NUM_INACTIVE_ACCS': 'Num', 
'NPD_NUM_NPD_SP_L18M': 'Num', 
'VM09_SP_VM1_04': 'Char', 
'AFF_T_07_01': 'Num', 
'TRD_B_53': 'Num', 
'NPD_NUM_EBAD_SP_L18M': 'Num', 
'VM03_SP_VM2_27': 'Num', 
'VM02_SP_VM1_18': 'Num', 
'SPA_B3_22': 'Num', 
'VM04_SP_VM1_21': 'Num', 
'VM04_SP_VM1_20': 'Num', 
'NPD_BAL_NPD_SP_L18M': 'Num', 
'SP_EDI_30': 'Num', 
'VM03_SP_VM2_28': 'Num', 
'TRD_A_17': 'Num', 
'VM10_SP_VM2_15': 'Num', 
'TRD_B_30': 'Num', 
'AFF_T_43': 'Num', 
'TRD_B_31': 'Num', 
'VM05_SP_VM1_19': 'Num', 
'AGE_MOST_RECENT': 'Num', 
'AFF_T_34_01': 'Num', 
'VM08_SP_VM2_14': 'Num', 
'TRD_C_05': 'Num', 
'ND_HAC_02': 'Num', 
'NDMAICH': 'Char', 
'VM09_SP_VM1_18': 'Num', 
'TRD_B_29': 'Num', 
'SP_J_61_01': 'Num', 
'SP_J_58_01': 'Num', 
'NPD_BAL_EBAD_SP_L18M': 'Num', 
'E1_B_08': 'Char', 
'VM08_SP_VM2_27': 'Num', 
'SP_EDI_22': 'Num', 
'VM05_SP_VM1_16': 'Num', 
'SP_EDI_31': 'Num', 
'VM10_SP_VM2_22': 'Num', 
'VM08_SP_VM2_18': 'Num', 
'TRD_A_21': 'Num', 
'TRD_A_18': 'Num', 
'VM08_SP_VM2_23': 'Num', 
'VM10_SP_VM2_27': 'Num', 
'VM08_SP_VM2_28': 'Num', 
'NUM_INACTIVE_ACCS_L36M': 'Num', 
'SPA_F3_36': 'Num', 
'E1_E_02': 'Num', 
'VM10_SP_VM2_23': 'Num', 
'SP_EDI_29': 'Num', 
'ND_HAC_03': 'Num', 
'E1_A_08': 'Num', 
'VM03_SP_VM2_24': 'Num', 
'TRD_STL_19': 'Num', 
'VM07_SP_VM1_26': 'Num', 
'VM04_SP_VM1_03': 'Char', 
'EA4_P_01': 'Char', 
'TRD_A_12': 'Num', 
'SP_I_66_01': 'Char', 
'TRD_STL_20': 'Num', 
'VM08_SP_VM2_07': 'Num', 
'VM01_SP_VM2_28': 'Num', 
'VM01_SP_VM2_29': 'Num', 
'VM04_SP_VM1_04': 'Char', 
'TRD_P_23': 'Num', 
'AFF_S_22_01': 'Char', 
'VM08_SP_VM2_17': 'Num', 
'AFF_C_06': 'Char', 
'GEN11_SP_J_58': 'Num', 
'E2_K_02': 'Num', 
'SP_B2_19': 'Num', 
'SP_EDI_12': 'Char', 
'VM03_SP_VM2_16': 'Num', 
'SP_EDI_19': 'Char', 
'VM10_SP_VM2_28': 'Num', 
'VM10_SP_VM2_29': 'Num', 
'AFF_V_19': 'Num', 
'SPA_A_01': 'Num', 
'AFF_S_03_01': 'Char', 
'VM04_SP_VM1_15': 'Num', 
'E1_B_13': 'Num', 
'VM08_SP_VM2_03': 'Char', 
'VM10_SP_VM2_16': 'Num', 
'VM04_SP_VM1_11': 'Num', 
'TRD_P_36': 'Num', 
'EA5_U_01': 'Num', 
'VM02_SP_VM1_05': 'Char', 
'GEN11_SP_L_81': 'Num', 
'HC_C_01': 'Num', 
'AFF_C_03_01': 'Char', 
'VM02_SP_VM1_11': 'Num', 
'VM01_SP_VM2_34': 'Num', 
'TRD_A_16': 'Num', 
'E2_J_02': 'Num', 
'VM01_SP_VM2_32': 'Num', 
'VM03_SP_VM2_07': 'Num', 
'VM08_SP_VM2_34': 'Num', 
'E2_H_01': 'Num', 
'VM01_SP_VM2_02': 'Char', 
'ND_ECC_05': 'Num', 
'AFF_M_01_01': 'Num', 
'TRD_O_08': 'Num', 
'VM05_SP_VM1_20': 'Num', 
'VM05_SP_VM1_21': 'Num', 
'NPD_NUM_NPD_SP_L12M': 'Num', 
'VM04_SP_VM1_26': 'Num', 
'NPD_NUM_EBAD_SP_L12M': 'Num', 
'PD_B_06': 'Num', 
'E2_H_02': 'Num', 
'VM04_SP_VM1_22': 'Num', 
'VM04_SP_VM1_16': 'Num', 
'TRD_A_23': 'Num', 
'AFF_T_82_01': 'Char', 
'VM04_SP_VM1_18': 'Num', 
'E1_B_12': 'Num', 
'VM09_SP_VM1_20': 'Num', 
'VM09_SP_VM1_21': 'Num', 
'TRD_RAG_13_AIQ2': 'Char', 
'VM08_SP_VM2_19': 'Num', 
'ND_PSD_05': 'Num', 
'SP_B1_15': 'Num', 
'TRD_A_19': 'Num', 
'EA2_Q_02': 'Num', 
'NPD_BAL_NPD_SP_L12M': 'Num', 
'ND_HAC_10': 'Num', 
'VM04_SP_VM1_01': 'Num', 
'GEN11_SP_L_78': 'Num', 
'SP_EDI_32': 'Num', 
'VM05_SP_VM1_04': 'Char', 
'AFF_V_09': 'Num', 
'ND_HAC_05': 'Num', 
'VM04_SP_VM1_08': 'Num', 
'NDMAUNSECARR': 'Char', 
'ND_ECC_01': 'Num', 
'E2_K_01': 'Num', 
'NO_CA_L3M': 'Num', 
'ND_ECC_10': 'Num', 
'VM04_SP_VM1_07': 'Num', 
'HC_B_01': 'Num', 
'E2_H_11': 'Num', 
'SPA_H_41': 'Num', 
'E2_G_09': 'Num', 
'ND_PSD_11': 'Num', 
'TRD_P_08': 'Num', 
'SP_J_60_01': 'Num', 
'SP_J_56_01': 'Num', 
'PD_F_21': 'Num', 
'VM04_SP_VM1_06': 'Num', 
'E1_B_04': 'Num', 
'SPA_E1_26': 'Num', 
'SP_C_24': 'Char', 
'VM10_SP_VM2_18': 'Num', 
'VM10_SP_VM2_30': 'Num', 
'NPD_BAL_EBAD_SP_L12M': 'Num', 
'AFF_V_17_01': 'Num', 
'AFF_T_56_01': 'Num', 
'SP_EDI_38': 'Num', 
'TRD_B_49': 'Num', 
'VM07_SP_VM1_10': 'Num', 
'E2_G_01': 'Num', 
'AFF_T_67_01': 'Char', 
'AFF_C_19_01': 'Char', 
'VM06_SP_VM1_05': 'Char', 
'NPD_NUM_NPD_SPA_L18M': 'Num', 
'NPD_NUM_EBAD_SPA_L18M': 'Num', 
'AFF_V_13_01': 'Num', 
'VM10_SP_VM2_07': 'Num', 
'TRD_RAG_13': 'Char', 
'ND_LNK_01': 'Char', 
'ND_ECC_03': 'Num', 
'VM01_SP_VM2_12': 'Num', 
'EA4_Q_06': 'Char', 
'VM03_SP_VM2_26': 'Num'} 

COLS_MAN_CORRECT: Dict[str, Dict[str, Any]] = dict()

MIN_IV: float = 0.1
CORR_PEARSON_CUT_OFF: float = 0.75
RANDOM_SEED: int = 42


TARGET_NAME: str = 'GB6_Flag_2Limit'
PALETTE_TARGET: Dict[str, str] = {
    'G': 'green',
    'B': 'red',
    'I': 'black'
}
TARGET_BIN: str = f'_{TARGET_NAME}_bin'
TARGET_BIN_IisB: str = f'_{TARGET_NAME}_bin_IisB'

PROJECT_PATH: str = os.path.join(os.getcwd(), '..')
DATA_PATH: str = os.path.join(PROJECT_PATH, 'data')
OPT_BIN_PATH: str = os.path.join(os.path.join(PROJECT_PATH, 'objects'), 'optbinners')

In [5]:
# read data
df: pd.DataFrame = pd.read_csv(os.path.join(DATA_PATH, 'final_aiq2.csv'))

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1622656214.py:2: DtypeWarning: Columns (310,1307,1349,1351,1352,1400,1401,1402,1403,1404,1405) have mixed types. Specify dtype option on import or set low_memory=False.
  df: pd.DataFrame = pd.read_csv(os.path.join(DATA_PATH, 'final_aiq2.csv'))


In [6]:
print('Number of rows: %.0f; columns: %.0f' % df.shape)

Number of rows: 110462; columns: 1774


In [7]:
#  check if all necessary columns are present
assert df[COLUMNS_DICT.keys()].shape[1] == len(COLUMNS_DICT)

In [8]:
TRAIN_BIN_FROM: datetime.date = pd.to_datetime('2021-07-01').date()
TRAIN_BIN_TO: datetime.date = pd.to_datetime('2021-12-01').date()

# Preprocessing

## Create the target 'I', 'B' are 1, 'G' - 0

In [9]:
warnings.filterwarnings("ignore", category=FutureWarning)
df[TARGET_BIN_IisB] = df[TARGET_NAME].replace(
    {
        'G': 0,
        'B': 1, 
        'I': 1
    }
)

In [10]:
# check the distribution
df.groupby(TARGET_NAME)[TARGET_BIN_IisB].value_counts()

GB6_Flag_2Limit  _GB6_Flag_2Limit_bin_IisB
B                1                             4449
G                0                            99444
I                1                             6569
Name: count, dtype: int64

In [11]:
# compare with already existed column
df.groupby(TARGET_NAME)[TARGET_BIN].value_counts()

GB6_Flag_2Limit  _GB6_Flag_2Limit_bin
B                1                        4449
G                0                       99444
I                0                        6569
Name: count, dtype: int64

## Convert dates to the corresponding types

In [12]:
for col in ['_RDATE_EOM', '_RDATE']:
    df[col] = pd.to_datetime(df[col]).dt.date

In [13]:
print('Available date range:')
df['_RDATE'].agg(['min', 'max'])

Available date range:


min    2021-06-30
max    2023-11-30
Name: _RDATE, dtype: object

## Dafine dataset to train OptBinnning on

In [14]:
df_train: pd.DataFrame = df[
    (df._RDATE >= TRAIN_BIN_FROM) &
    (df._RDATE < TRAIN_BIN_TO) & 
    (df[TARGET_NAME] != 'I')
].copy()

In [15]:
print('Number of rows: %.0f; columns: %.0f' % df_train.shape)

Number of rows: 13940; columns: 1775


In [16]:
print('Target population of the selected period:')
df_train.groupby(['_RDATE_EOM'])[TARGET_NAME].value_counts(sort=False).sort_index().rename('n')

Target population of the selected period:


_RDATE_EOM  GB6_Flag_2Limit
2021-07-31  B                    65
            G                  2348
2021-08-31  B                    66
            G                  2383
2021-09-30  B                    76
            G                  2657
2021-10-31  B                   131
            G                  3054
2021-11-30  B                    93
            G                  3067
Name: n, dtype: int64

In [17]:
df_train[TARGET_BIN_IisB].value_counts()

_GB6_Flag_2Limit_bin_IisB
0    13509
1      431
Name: count, dtype: int64

# Binning

## Numeric columns

Negatives are treated as special categories; specified columns have int edges;

In [18]:
COLUMNS_INT_EDGES = [
    'VM01_SP_VM2_15', 
    'TRD_A_08', 
    'SP_G_37', 
    'TRD_C_07', 
    'SPA_F2_33', 
    'VM01_SP_VM2_14',
    'VM01_SP_VM2_24', 
    'SP_G_38', 
    'TRD_O_05', 
    'E1_A_09', 
    'TRD_O_07', 
    'ND_INC_03',
    'VM01_SP_VM2_17',
    'SP_B1_14',
    'GEN11_SP_N_91',
    'GEN11_SP_EDI_07',
    'ND_ECC_06', 
    'EA5_S_01',
    'CLU_CLI_L6M_NPR_L6M',
    'E1_B_06',
    'SP_I_63_01',
    'E4_Q_04', 
    'VM08_SP_VM2_15',
    'VAL_DELINQ_ACCS',
    'VM08_SP_VM2_20',
    'EA1_D_02',
    'AFF_E_07_01',
    'TRD_O_06', 
    'E2_G_08',
    'AFF_T_07_01',
    'VM02_SP_VM1_18',
    'E2_G_08',
    'AFF_T_07_01'
]

In [19]:
for col in COLUMNS_INT_EDGES:
    print(col, '\n')
    print(np.sort(df[col].unique()))
    print('-'*20)

VM01_SP_VM2_15 

[-2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21
 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45
 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93
 94 95 96 97 98 99]
--------------------
TRD_A_08 

[-2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21
 22 23 24 25 27 28 29 30 31 32 37 40 44 47]
--------------------
SP_G_37 

[ -2  -1   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 

In [20]:
exceptions = []
for col_name, col_type in tqdm(COLUMNS_DICT.items()):
    try: 
        unique_vals = df[col_name].unique()
        special_cases: List[Any] = []
        dict_to_replace: Dict[str, float] = dict()
        unique_floats: List[float] = []
        for elem in unique_vals:
            try:
                dict_to_replace[elem] = float(elem)
                unique_floats += [float(elem)]
            except ValueError:
                special_cases += [elem] 
        
        unique_floats = np.array(unique_floats) 
        special_cases += list(unique_floats[unique_floats < 0])
        if_numeric = col_type.lower().strip() == 'num'
        if len(special_cases) > 0:
            special_cases = dict(zip(special_cases, special_cases))
        
        if if_numeric:
            optb = OptimalBinning(
                name=col_name, 
                dtype='numerical', 
                solver="mip",
                special_codes=special_cases if len(special_cases) > 0 else None,
                monotonic_trend='auto_asc_desc', 
                split_digits=0 if col_name in COLUMNS_INT_EDGES else None
            )
            optb.fit(df_train[col_name].replace(dict_to_replace).values, df_train[TARGET_BIN_IisB].values)
            df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
            df[f'Bin{col_name}'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="bins")
            dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
            optbins_params = optb.get_params()
            dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-excl.jblb'))
    except Exception as e:
        exceptions += [(col_name, e)]

 16%|█████████████▎                                                                   | 47/287 [01:47<02:33,  1.57it/s]C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1932735252.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1932735252.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name}'] = optb.transform(df[col_name].replace(dict_to_replace).values

In [21]:
print('If any Exception: ', len(exceptions))

If any Exception:  2


In [22]:
print('Number of rows: %.0f; columns: %.0f' % df.shape)

Number of rows: 110462; columns: 2279


In [23]:
# list columns which weren't transformed
if exceptions != []:
    exceptions_cols = [elem[0] for elem in exceptions]
    print(exceptions_cols)

['SP_EDI_37', 'SP_EDI_38']


In [24]:
for col in exceptions_cols:
    print(col, ': ', COLUMNS_DICT[col].lower().strip())
    print('Unique: ', df[col].unique())
    print('-'*50)

SP_EDI_37 :  num
Unique:  ['D' 'C' 'H' 'B' 'G' 'X' 'E' 'F' 'A' 'T' 'I']
--------------------------------------------------
SP_EDI_38 :  num
Unique:  ['B' 'X' 'T' 'G' 'E' 'D']
--------------------------------------------------


In [25]:
# Columns with only categorical values are defined as Num. The default `Num` type is changed to `Char`
for col in exceptions_cols:
    COLUMNS_DICT[col] = 'Char'

## Categorical columns

In [26]:
exceptions_cat = []
for col_name, col_type in tqdm(COLUMNS_DICT.items()):
    try: 
        unique_vals = df[col_name].unique()
        special_cases: List[Any] = []
        dict_to_replace: Dict[str, float] = dict()
        unique_floats: List[float] = []
        for elem in unique_vals:
            try:
                dict_to_replace[elem] = float(elem)
                unique_floats += [float(elem)]
            except ValueError:
                pass

        unique_floats = np.array(unique_floats)
        special_cases += list(unique_floats[unique_floats < 0])

        if len(special_cases) > 0:
            special_cases = dict(zip(special_cases, special_cases))
        
        if_numeric = col_type.lower().strip() == 'num'
        if not if_numeric:
            optb = OptimalBinning(
                name=col_name, 
                dtype='categorical', 
                solver="mip",
                special_codes=special_cases if len(special_cases) > 0 else None
            )
            optb.fit(df_train[col_name].replace(dict_to_replace).values, df_train[TARGET_BIN_IisB].values)
            df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
            df[f'Bin{col_name}'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="bins")
            dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
            optbins_params = optb.get_params()
            dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-excl.jblb'))
    except Exception as e:
        exceptions_cat += [(col_name, e)]

  0%|                                                                                          | 0/287 [00:00<?, ?it/s]C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2365401015.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2365401015.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name}'] = optb.transform(df[col_name].replace(dict_to_replace).values

In [27]:
print('If any Exception: ', len(exceptions_cat))

If any Exception:  0


In [28]:
print('Number of rows: %.0f; columns: %.0f' % df.shape)

Number of rows: 110462; columns: 2349


## Modification of splits of several columns

### Regroupping

#### E4_Q_17

In [29]:
col_name='E4_Q_17'

Ok given the low volume of the in  the arrears categories and due to the unintuitive pattern (i.e. reversals) if you were to categorise the data in any other way I'd recommend the following groupings: 

1) On ER and Worst CAIS 0/U (i.e. bin 2 on its own),
2) Not on ER and worst CAIS 0/U (i.e. bin 6 on its own)
3) Both the T/N categories (i.e. bins 1 and 5 grouped together)
4) then bins 3,4,7 and 8 together.

This will give you 4 robust groups with adequate volumes in each category and an intuitive pattern 

In [30]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={2: 2, '(1, 5)': [1, 5], '(3, 4, 7, 8)': [3, 4, 7, 8]}
)
optb.fit(df_train[col_name].astype(int), df_train[TARGET_BIN_IisB].values)
df[col_name + 'OptBin'] = optb.transform(df[col_name].astype(int).values, metric="woe")
df[f'Bin{col_name}'] = optb.transform(df[col_name].astype(int).values, metric="bins")
dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-excl.jblb'))
optb.binning_table.build()

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[6],2744,0.196844,2648,96,0.034985,-0.127792,0.003415,0.000427
1,2,8286,0.594405,8180,106,0.012793,0.901005,0.323986,0.039182
2,"(1, 5)",243,0.017432,229,14,0.057613,-0.650339,0.010100,0.001241
3,"(3, 4, 7, 8)",2667,0.191320,2452,215,0.080615,-1.010982,0.320816,0.038477
4,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.658317,0.079326


#### TRD_A_20

In [31]:
np.sort(df.TRD_A_20.unique())

array([-3, -2, -1,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
       15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 34, 35, 36], dtype=int64)

In [32]:
col_name='TRD_A_20'

In [33]:
#  change values >= 1 to '1+' category
# - train dataset
mask = (df_train[col_name] >= 1)
df_train[col_name+'OptBin'] = df_train[col_name].mask(mask, '1+').values
del mask

# - full dataset
mask = df[col_name] >= 1
df[col_name+'OptBin'] = df[col_name].mask(mask, '1+').values

In [34]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'{-3, 0}': [-3, 0], -2:-2, -1:-1}, 
    split_digits=0
)
optb.fit(df_train[col_name+'OptBin'], df_train[TARGET_BIN_IisB].values)

df[f'Bin{col_name}'] = optb.transform(df[col_name+'OptBin'].values, metric="bins")
df[col_name + 'OptBin'] = optb.transform(df[col_name+'OptBin'].values, metric="woe")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-excl.jblb'))
optb.binning_table.build()

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[1+],465,0.033357,457,8,0.017204,0.600239,0.009164,0.001129
1,"{-3, 0}",5977,0.428766,5932,45,0.007529,1.436451,0.480789,0.055412
2,-2,7457,0.534935,7079,378,0.050691,-0.51501,0.181803,0.022478
3,-1,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
4,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.671757,0.079018


In [35]:
COLS_MAN_CORRECT[col_name] = [
    {'Bin': '[1+]', 'WoE': 0}
]

In [36]:
COLS_MAN_CORRECT

{'TRD_A_20': [{'Bin': '[1+]', 'WoE': 0}]}

In [37]:
# Correct WoE values: 
df.loc[df[f'Bin{col_name}'] == COLS_MAN_CORRECT[col_name][0]['Bin'], col_name + 'OptBin'] = COLS_MAN_CORRECT[col_name][0]['WoE']

#### GEN11_SP_N_92

In [38]:
col_name='GEN11_SP_N_92'

In [39]:
df[col_name].value_counts(normalize=True)

GEN11_SP_N_92
BB    0.122974
GB    0.101809
G     0.101573
P     0.096793
PB    0.093553
B     0.060220
JB    0.044884
J     0.044350
F     0.042494
FB    0.036447
A     0.034971
S     0.028037
SB    0.024732
I     0.021175
4B    0.020278
IB    0.020088
AA    0.018350
1B    0.013579
3B    0.012067
GA    0.007976
6B    0.007532
MB    0.006446
UB    0.004499
5B    0.004373
U     0.003829
JA    0.003685
PA    0.003675
FA    0.003648
T     0.003159
SA    0.002526
IA    0.002318
4A    0.001557
M     0.001385
2B    0.001340
1A    0.001005
3A    0.000987
6A    0.000733
5A    0.000435
UA    0.000281
2A    0.000118
MA    0.000118
Name: proportion, dtype: float64

##### GEN11_SP_N_92_IsA

In [40]:
col_name='GEN11_SP_N_92'

In [41]:
df_train[col_name + '_IsA'] = df_train[col_name].str.contains('A').astype(int).values
df[col_name + '_IsA'] = df[col_name].str.contains('A').astype(int).values

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2528358506.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_IsA'] = df[col_name].str.contains('A').astype(int).values


In [42]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip"
)
optb.fit(df_train[col_name + '_IsA'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_IsA' + 'OptBin'] = optb.transform(df[col_name + '_IsA'].values, metric="woe")
df[f'Bin{col_name + '_IsA'}'] = optb.transform(df[col_name + '_IsA'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_IsA'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_IsA'}-negatives-separately-I-excl.jblb'))
df_isa = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1176059042.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_IsA' + 'OptBin'] = optb.transform(df[col_name + '_IsA'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1176059042.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_IsA'}'] = optb.transform(df[col_name + '_IsA'].values, metric="bins")


##### GEN11_SP_N_92_IsB

In [43]:
col_name='GEN11_SP_N_92'

In [44]:
df_train[col_name + '_IsB'] = df_train[col_name].str.contains('B').astype(int).values
df[col_name + '_IsB'] = df[col_name].str.contains('B').astype(int).values

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\636956716.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_IsB'] = df[col_name].str.contains('B').astype(int).values


In [45]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip"
)
optb.fit(df_train[col_name + '_IsB'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_IsB' + 'OptBin'] = optb.transform(df[col_name + '_IsB'].values, metric="woe")
df[f'Bin{col_name + '_IsB'}'] = optb.transform(df[col_name + '_IsB'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_IsB'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_IsB'}-negatives-separately-I-excl.jblb'))
df_isb = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1686152275.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_IsB' + 'OptBin'] = optb.transform(df[col_name + '_IsB'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1686152275.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_IsB'}'] = optb.transform(df[col_name + '_IsB'].values, metric="bins")


##### GEN11_SP_N_92_ABOther

In [46]:
col_name='GEN11_SP_N_92'

In [47]:
print('If any case with "A" and "B" at the same value: ', (df[col_name].str.contains('A') & df[col_name].str.contains('B')).any())

If any case with "A" and "B" at the same value:  False


If `GEN11_SP_N_92` contains 
- 'A' -> 'A' 
- 'B' -> 'B' 
- no 'A', no 'B' -> 'Other' 

In [48]:
df_train[col_name + '_ABOther'] = 'Other'

df_train[col_name + '_ABOther'] = df_train[col_name + '_ABOther'].mask(df_train[col_name].str.contains('A'), 'A').values
df_train[col_name + '_ABOther'] = df_train[col_name + '_ABOther'].mask(df_train[col_name].str.contains('B'), 'B').values

In [49]:
df[col_name + '_ABOther'] = 'Other'


df[col_name + '_ABOther'] = df[col_name + '_ABOther'].mask(df[col_name].str.contains('A'), 'A').values
df[col_name + '_ABOther'] = df[col_name + '_ABOther'].mask(df[col_name].str.contains('B'), 'B').values

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\425197986.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_ABOther'] = 'Other'


In [50]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
)
optb.fit(df_train[col_name + '_ABOther'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_ABOther' + 'OptBin'] = optb.transform(df[col_name + '_ABOther'].values, metric="woe")
df[f'Bin{col_name + '_ABOther'}'] = optb.transform(df[col_name + '_ABOther'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_ABOther'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_ABOther'}-negatives-separately-I-excl.jblb'))
df_abother = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\487758087.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_ABOther' + 'OptBin'] = optb.transform(df[col_name + '_ABOther'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\487758087.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_ABOther'}'] = optb.transform(df[col_name + '_ABOther'].values, metric="bins")


In [51]:
df[df[col_name + '_ABOther'] == 'Other'][col_name].value_counts() * 100 / df.shape[0]

GEN11_SP_N_92
G    10.157339
P     9.679347
J     4.435009
F     4.249425
S     2.803679
I     2.117470
U     0.382937
T     0.315946
M     0.138509
Name: count, dtype: float64

##### Select one of `_IsA`, `_IsB`, `_ABOther`

In [52]:
df[[col_name + '_ABOtherOptBin', col_name + '_IsAOptBin',  col_name + '_IsBOptBin']].corr()

,GEN11_SP_N_92_ABOtherOptBin,GEN11_SP_N_92_IsAOptBin,GEN11_SP_N_92_IsBOptBin
GEN11_SP_N_92_ABOtherOptBin,1.000000,0.463414,0.992074
GEN11_SP_N_92_IsAOptBin,0.463414,1.000000,0.348389
GEN11_SP_N_92_IsBOptBin,0.992074,0.348389,1.000000


In [53]:
for sample, name in zip([df_isa, df_isb, df_abother], ['IsA', 'IsB', 'ABOther']):
    print(name, ': ', sample.loc['Totals', 'IV'])

IsA :  0.12958615138494695
IsB :  0.5615451158004616
ABOther :  0.5753956720536564


`ABOther`  has the largest `IV`. It's going to be left.

In [54]:
del df[col_name + '_IsA']
del df[col_name + '_IsA' + 'OptBin'], df[f'Bin{col_name + '_IsA'}']

In [55]:
os.remove(os.path.join(OPT_BIN_PATH, f'{col_name + '_IsA'}-negatives-separately-I-excl.jblb'))
os.remove(os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_IsA'}-negatives-separately-I-excl.jblb'))

In [56]:
del df[col_name + '_IsB']
del df[col_name + '_IsB' + 'OptBin'], df[f'Bin{col_name + '_IsB'}']

In [57]:
os.remove(os.path.join(OPT_BIN_PATH, f'{col_name + '_IsB'}-negatives-separately-I-excl.jblb'))
os.remove(os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_IsB'}-negatives-separately-I-excl.jblb'))

In [58]:
COLUMNS_DICT.update({'GEN11_SP_N_92_ABOther': 'Char'})

##### GEN11_SP_N_92_NotAB

In [59]:
col_name='GEN11_SP_N_92'

In [60]:
df_train[col_name + '_NotAB'] = df_train[col_name].copy()


df_train[col_name + '_NotAB'] = df_train[col_name + '_NotAB'].mask(df_train[col_name].str.contains('A'), 'A').values
df_train[col_name + '_NotAB'] = df_train[col_name + '_NotAB'].mask(df_train[col_name].str.contains('B'), 'B').values
df_train[col_name + '_NotAB'].value_counts(normalize=True)

GEN11_SP_N_92_NotAB
B    0.606313
G    0.094548
A    0.077690
P    0.076255
J    0.048996
F    0.038164
S    0.027905
I    0.022812
T    0.002941
U    0.002296
M    0.002080
Name: proportion, dtype: float64

In [61]:
df[col_name + '_NotAB'] = df[col_name].copy()


df[col_name + '_NotAB'] = df[col_name + '_NotAB'].mask(df[col_name].str.contains('A'), 'A').values
df[col_name + '_NotAB'] = df[col_name + '_NotAB'].mask(df[col_name].str.contains('B'), 'B').values
df[col_name + '_NotAB'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\3454252018.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_NotAB'] = df[col_name].copy()


GEN11_SP_N_92_NotAB
B    0.574822
G    0.101573
P    0.096793
A    0.082381
J    0.044350
F    0.042494
S    0.028037
I    0.021175
U    0.003829
T    0.003159
M    0.001385
Name: proportion, dtype: float64

In [62]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'A': 'A', 'B': 'B', 'T': 'T', 'M': 'M'}
)
optb.fit(df_train[col_name + '_NotAB'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_NotAB' + 'OptBin'] = optb.transform(df[col_name + '_NotAB'].values, metric="woe")
df[f'Bin{col_name + '_NotAB'}'] = optb.transform(df[col_name + '_NotAB'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_NotAB'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_NotAB'}-negatives-separately-I-excl.jblb'))
df_NotAB = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2408014883.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_NotAB' + 'OptBin'] = optb.transform(df[col_name + '_NotAB'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2408014883.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_NotAB'}'] = optb.transform(df[col_name + '_NotAB'].values, metric="bins")


In [63]:
df_NotAB

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"[F, G]",1850,0.132712,1763,87,0.047027,-0.436139,0.031119,0.003859
1,[P],1063,0.076255,1006,57,0.053622,-0.574317,0.033185,0.004092
2,"[S, I]",707,0.050717,663,44,0.062235,-0.732418,0.038825,0.004748
3,"[J, U]",715,0.051291,665,50,0.069930,-0.857239,0.057249,0.006945
4,A,1083,0.077690,1000,83,0.076639,-0.956089,0.113345,0.013652
5,B,8452,0.606313,8342,110,0.013015,0.883575,0.320114,0.038761
6,T,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
7,M,29,0.002080,29,0,0.000000,0.0,0.000000,0.000000
8,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.593837,0.072057


In [64]:
df[[col_name + '_NotABOptBin', col_name + '_ABOtherOptBin']].corr()

,GEN11_SP_N_92_NotABOptBin,GEN11_SP_N_92_ABOtherOptBin
GEN11_SP_N_92_NotABOptBin,1.000000,0.720475
GEN11_SP_N_92_ABOtherOptBin,0.720475,1.000000


In [65]:
print(df_NotAB.loc['Totals', 'IV'])
print(df_abother.loc['Totals', 'IV'])

0.5938365801424466
0.5753956720536564


In [66]:
COLUMNS_DICT.update({'GEN11_SP_N_92_NotAB': 'Char'})

##### GEN11_SP_N_92_IfSoleJointOtherAcc

In [67]:
col_name='GEN11_SP_N_92'

In [68]:
df[df[col_name].str.contains(r'\d')][col_name].unique()

array(['1B', '3B', '6B', '6A', '5B', '4B', '3A', '2B', '4A', '1A', '2A',
       '5A'], dtype=object)

In [69]:
df[~df[col_name].str.contains(r'\d')][col_name].unique()

array(['GB', 'P', 'BB', 'AA', 'A', 'JB', 'IB', 'J', 'SB', 'G', 'PB', 'T',
       'FB', 'F', 'B', 'S', 'I', 'MB', 'JA', 'M', 'GA', 'PA', 'U', 'SA',
       'FA', 'UB', 'UA', 'IA', 'MA'], dtype=object)

In [70]:
df[df[col_name].str.contains(r'[SUFGIJ]+')][col_name].unique()

array(['GB', 'JB', 'IB', 'J', 'SB', 'G', 'FB', 'F', 'S', 'I', 'JA', 'GA',
       'U', 'SA', 'FA', 'UB', 'UA', 'IA'], dtype=object)

In [71]:
df[~df[col_name].str.contains(r'[SUFGIJ]+')][col_name].unique()

array(['P', 'BB', 'AA', 'A', '1B', 'PB', '3B', 'T', '6B', '6A', 'B', '5B',
       '4B', '3A', 'MB', '2B', 'M', '4A', 'PA', '1A', '2A', '5A', 'MA'],
      dtype=object)

In [72]:
df_train[col_name + '_IfSoleJointOtherAcc'] = 'Other'


df_train[col_name + '_IfSoleJointOtherAcc'] = df_train[col_name + '_IfSoleJointOtherAcc'].mask(df[col_name].str.contains(r'[SUFGIJ]+'), 'Sole').values
df_train[col_name + '_IfSoleJointOtherAcc'] = df_train[col_name + '_IfSoleJointOtherAcc'].mask(df_train[col_name].str.contains(r'\d'), 'Joint').values
df_train[col_name + '_IfSoleJointOtherAcc'].value_counts(normalize=True)

GEN11_SP_N_92_IfSoleJointOtherAcc
Sole     0.514204
Other    0.410258
Joint    0.075538
Name: proportion, dtype: float64

In [73]:
df[col_name + '_IfSoleJointOtherAcc'] = 'Other'


df[col_name + '_IfSoleJointOtherAcc'] = df[col_name + '_IfSoleJointOtherAcc'].mask(df[col_name].str.contains(r'[SUFGIJ]+'), 'Sole').values
df[col_name + '_IfSoleJointOtherAcc'] = df[col_name + '_IfSoleJointOtherAcc'].mask(df[col_name].str.contains(r'\d'), 'Joint').values
df[col_name + '_IfSoleJointOtherAcc'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2388596109.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_IfSoleJointOtherAcc'] = 'Other'


GEN11_SP_N_92_IfSoleJointOtherAcc
Sole     0.494351
Other    0.441645
Joint    0.064004
Name: proportion, dtype: float64

In [74]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
)
optb.fit(df_train[col_name + '_IfSoleJointOtherAcc'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_IfSoleJointOtherAcc' + 'OptBin'] = optb.transform(df[col_name + '_IfSoleJointOtherAcc'].values, metric="woe")
df[f'Bin{col_name + '_IfSoleJointOtherAcc'}'] = optb.transform(df[col_name + '_IfSoleJointOtherAcc'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_IfSoleJointOtherAcc'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_IfSoleJointOtherAcc'}-negatives-separately-I-excl.jblb'))
df_IfJointAcc = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1944781999.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_IfSoleJointOtherAcc' + 'OptBin'] = optb.transform(df[col_name + '_IfSoleJointOtherAcc'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1944781999.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_IfSoleJointOtherAcc'}'] = optb.transform(df[col_name + '_IfSoleJointOtherAcc'].values, metric="bins")


In [75]:
df_IfJointAcc

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[Joint],1053,0.075538,1038,15,0.014245,0.791998,0.033292,0.004056
1,[Other],5719,0.410258,5550,169,0.029551,0.046651,0.000874,0.000109
2,[Sole],7168,0.514204,6921,247,0.034459,-0.112076,0.006810,0.000851
3,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
4,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.040975,0.005016


In [76]:
# correlation of the GEN11_SP_N_92-based features
df[[col_name + '_ABOtherOptBin', col_name + '_NotABOptBin', col_name + '_IfSoleJointOtherAccOptBin']].corr()

,GEN11_SP_N_92_ABOtherOptBin,GEN11_SP_N_92_NotABOptBin,GEN11_SP_N_92_IfSoleJointOtherAccOptBin
GEN11_SP_N_92_ABOtherOptBin,1.000000,0.720475,0.21286
GEN11_SP_N_92_NotABOptBin,0.720475,1.000000,0.25994
GEN11_SP_N_92_IfSoleJointOtherAccOptBin,0.212860,0.259940,1.00000


In [77]:
# IV of the GEN11_SP_N_92-based features
print('ABOther: ', df_abother.loc['Totals', 'IV'])
print('NotAB: ', df_NotAB.loc['Totals', 'IV'])
print('If Sole/Joint/Other Acc: ', df_IfJointAcc.loc['Totals', 'IV'])

ABOther:  0.5753956720536564
NotAB:  0.5938365801424466
If Sole/Joint/Other Acc:  0.04097493047046003


In [78]:
COLUMNS_DICT.update({'GEN11_SP_N_92_IfSoleJointOtherAcc': 'Char'})

##### GEN11_SP_N_92_ByTurnover

In [79]:
col_name='GEN11_SP_N_92'

In [80]:
df[df[col_name].str.contains(r'[SU]+[12]?')][col_name].unique()

array(['SB', 'S', 'U', 'SA', 'UB', 'UA'], dtype=object)

In [81]:
df[df[col_name].str.contains(r'[SU]?[12]+')][col_name].unique()

array(['1B', '2B', '1A', '2A'], dtype=object)

In [82]:
df_train[col_name + '_ByTurnover'] = 'Other'


df_train[col_name + '_ByTurnover'] = df_train[col_name + '_ByTurnover'].mask(df_train[col_name].str.contains(r'[SU]+[12]?') | df_train[col_name].str.contains(r'[SU]?[12]+'), 'Stable').values
df_train[col_name + '_ByTurnover'] = df_train[col_name + '_ByTurnover'].mask(df_train[col_name].str.contains(r'[FG]+[34]?') | df_train[col_name].str.contains(r'[FG]?[34]+'), 'Fluctuating').values
df_train[col_name + '_ByTurnover'] = df_train[col_name + '_ByTurnover'].mask(df_train[col_name].str.contains(r'[IG]+[56]?') | df_train[col_name].str.contains(r'[IG]?[56]+'), 'Irregular').values
df_train[col_name + '_ByTurnover'].value_counts(normalize=True)

GEN11_SP_N_92_ByTurnover
Other          0.522382
Irregular      0.276255
Fluctuating    0.122669
Stable         0.078694
Name: proportion, dtype: float64

In [83]:
df[col_name + '_ByTurnover'] = 'Other'


df[col_name + '_ByTurnover'] = df[col_name + '_ByTurnover'].mask(df[col_name].str.contains(r'[SU]+[12]?') | df[col_name].str.contains(r'[SU]?[12]+'), 'Stable').values
df[col_name + '_ByTurnover'] = df[col_name + '_ByTurnover'].mask(df[col_name].str.contains(r'[FG]+[34]?') | df[col_name].str.contains(r'[FG]?[34]+'), 'Fluctuating').values
df[col_name + '_ByTurnover'] = df[col_name + '_ByTurnover'].mask(df[col_name].str.contains(r'[IG]+[56]?') | df[col_name].str.contains(r'[IG]?[56]+'), 'Irregular').values
df[col_name + '_ByTurnover'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\278762637.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_ByTurnover'] = 'Other'


GEN11_SP_N_92_ByTurnover
Other          0.534564
Irregular      0.268011
Fluctuating    0.117479
Stable         0.079946
Name: proportion, dtype: float64

In [84]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
)
optb.fit(df_train[col_name + '_ByTurnover'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_ByTurnover' + 'OptBin'] = optb.transform(df[col_name + '_ByTurnover'].values, metric="woe")
df[f'Bin{col_name + '_ByTurnover'}'] = optb.transform(df[col_name + '_ByTurnover'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_ByTurnover'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_ByTurnover'}-negatives-separately-I-excl.jblb'))
df_bt = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1753690109.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_ByTurnover' + 'OptBin'] = optb.transform(df[col_name + '_ByTurnover'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1753690109.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_ByTurnover'}'] = optb.transform(df[col_name + '_ByTurnover'].values, metric="bins")


In [85]:
df_bt

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[Fluctuating],1710,0.122669,1663,47,0.027485,0.121228,0.001704,0.000213
1,[Irregular],3851,0.276255,3740,111,0.028824,0.072307,0.001396,0.000175
2,[Other],7282,0.522382,7048,234,0.032134,-0.039825,0.000844,0.000106
3,[Stable],1097,0.078694,1058,39,0.035552,-0.144429,0.001758,0.000220
4,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
5,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.005702,0.000712


##### GEN11_SP_N_92_ByRP

In [86]:
col_name='GEN11_SP_N_92'

In [87]:
df[df[col_name].str.contains(r'[SFI]?')][col_name].unique()

array(['GB', 'P', 'BB', 'AA', 'A', 'JB', 'IB', 'J', 'SB', '1B', 'G', 'PB',
       '3B', 'T', 'FB', '6B', 'F', '6A', 'B', 'S', 'I', '5B', '4B', '3A',
       'MB', 'JA', '2B', 'M', '4A', 'GA', 'PA', 'U', 'SA', 'FA', 'UB',
       '1A', 'UA', '2A', 'IA', '5A', 'MA'], dtype=object)

In [88]:
df[df[col_name].str.contains(r'[UGJ]+')][col_name].unique()

array(['GB', 'JB', 'J', 'G', 'JA', 'GA', 'U', 'UB', 'UA'], dtype=object)

In [89]:
df[df[col_name].str.contains(r'[135]+')][col_name].unique()

array(['1B', '3B', '5B', '3A', '1A', '5A'], dtype=object)

In [90]:
df[df[col_name].str.contains(r'[246]+')][col_name].unique()

array(['6B', '6A', '4B', '2B', '4A', '2A'], dtype=object)

In [91]:
q1 = df[col_name].str.contains(r'[SFI]+')
q2 = df[col_name].str.contains(r'[UGJ]+')
q3 = df[col_name].str.contains(r'[135]+')
q4 = df[col_name].str.contains(r'[246]+')

In [92]:
df_train[col_name + '_ByRP'] = 'Other'


df_train[col_name + '_ByRP'] = df_train[col_name + '_ByRP'].mask(df_train[col_name].str.contains(r'[SFI]+'), 'SR').values
df_train[col_name + '_ByRP'] = df_train[col_name + '_ByRP'].mask(df_train[col_name].str.contains(r'[UGJ]+'), 'SNR').values
df_train[col_name + '_ByRP'] = df_train[col_name + '_ByRP'].mask(df_train[col_name].str.contains(r'[135]+'), 'JR').values
df_train[col_name + '_ByRP'] = df_train[col_name + '_ByRP'].mask(df_train[col_name].str.contains(r'[246]+'), 'JNR').values
df_train[col_name + '_ByRP'].value_counts(normalize=True)

GEN11_SP_N_92_ByRP
Other    0.410258
SNR      0.327188
SR       0.187016
JNR      0.040316
JR       0.035222
Name: proportion, dtype: float64

In [93]:
print((q1 & q2 & q3 & q4).any())

print((q1 & q2 & q3).any())
print((q1 & q2 & q4).any())
print((q1 & q3 & q4).any())
print((q2 & q3 & q4).any())

print((q1 & q2).any())
print((q1 & q3).any())
print((q1 & q4).any())
print((q2 & q3).any())
print((q2 & q4).any())
print((q3 & q4).any())

False
False
False
False
False
False
False
False
False
False
False


In [94]:
df[col_name + '_ByRP'] = 'Other'


df[col_name + '_ByRP'] = df[col_name + '_ByRP'].mask(df[col_name].str.contains(r'[SFI]+'), 'SR').values
df[col_name + '_ByRP'] = df[col_name + '_ByRP'].mask(df[col_name].str.contains(r'[UGJ]+'), 'SNR').values
df[col_name + '_ByRP'] = df[col_name + '_ByRP'].mask(df[col_name].str.contains(r'[135]+'), 'JR').values
df[col_name + '_ByRP'] = df[col_name + '_ByRP'].mask(df[col_name].str.contains(r'[246]+'), 'JNR').values
df[col_name + '_ByRP'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\3906090664.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_ByRP'] = 'Other'


GEN11_SP_N_92_ByRP
Other    0.441645
SNR      0.312886
SR       0.181465
JR       0.032446
JNR      0.031558
Name: proportion, dtype: float64

In [95]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'[JR]': 'JR', '[JNR]': 'JNR'}
)
optb.fit(df_train[col_name + '_ByRP'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_ByRP' + 'OptBin'] = optb.transform(df[col_name + '_ByRP'].values, metric="woe")
df[f'Bin{col_name + '_ByRP'}'] = optb.transform(df[col_name + '_ByRP'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_ByRP'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_ByRP'}-negatives-separately-I-excl.jblb'))
df_brp = optb.binning_table.build()
df_brp

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\3044351483.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_ByRP' + 'OptBin'] = optb.transform(df[col_name + '_ByRP'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\3044351483.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_ByRP'}'] = optb.transform(df[col_name + '_ByRP'].values, metric="bins")


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[Other],5719,0.410258,5550,169,0.029551,0.046651,0.000874,0.000109
1,[SNR],4561,0.327188,4410,151,0.033107,-0.070653,0.001689,0.000211
2,[SR],2607,0.187016,2511,96,0.036824,-0.180915,0.006669,0.000832
3,[JR],491,0.035222,486,5,0.010183,1.131767,0.027587,0.003275
4,[JNR],562,0.040316,552,10,0.017794,0.56596,0.009995,0.001233
5,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.046813,0.005661


In [96]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'[SR, SNR]': ['SR', 'SNR'], '[JNR]': 'JNR', '[JR]': 'JR'}
)
optb.fit(df_train[col_name + '_ByRP'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_ByRP' + 'OptBin'] = optb.transform(df[col_name + '_ByRP'].values, metric="woe")
df[f'Bin{col_name + '_ByRP'}'] = optb.transform(df[col_name + '_ByRP'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_ByRP'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_ByRP'}-negatives-separately-I-excl.jblb'))
df_brp = optb.binning_table.build()
df_brp

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[Other],5719,0.410258,5550,169,0.029551,0.046651,0.000874,0.000109
1,"[SR, SNR]",7168,0.514204,6921,247,0.034459,-0.112076,0.006810,0.000851
2,[JNR],562,0.040316,552,10,0.017794,0.56596,0.009995,0.001233
3,[JR],491,0.035222,486,5,0.010183,1.131767,0.027587,0.003275
4,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.045265,0.005468


In [97]:
df[col_name].map(len).max()

2

In [98]:
print(df_IfJointAcc.loc['Totals', 'IV'])
print(df_bt.loc['Totals', 'IV'])
print(df_brp.loc['Totals', 'IV'])

0.04097493047046003
0.005701925589184631
0.04526506802478246


In [99]:
df[[col_name +  '_IfSoleJointOtherAccOptBin', col_name + '_ByTurnover' + 'OptBin', col_name + '_ByRP' + 'OptBin']].corr()

,GEN11_SP_N_92_IfSoleJointOtherAccOptBin,GEN11_SP_N_92_ByTurnoverOptBin,GEN11_SP_N_92_ByRPOptBin
GEN11_SP_N_92_IfSoleJointOtherAccOptBin,1.000000,-0.018929,0.126108
GEN11_SP_N_92_ByTurnoverOptBin,-0.018929,1.000000,-0.480815
GEN11_SP_N_92_ByRPOptBin,0.126108,-0.480815,1.000000


In [100]:
df[[col for col in df.columns if (col_name in col) & ('OptBin' in col)]].corr()

,GEN11_SP_N_92_ABOtherOptBin,GEN11_SP_N_92_NotABOptBin,GEN11_SP_N_92_IfSoleJointOtherAccOptBin,GEN11_SP_N_92_ByTurnoverOptBin,GEN11_SP_N_92_ByRPOptBin
GEN11_SP_N_92_ABOtherOptBin,1.000000,0.720475,0.212860,-0.038274,0.092571
GEN11_SP_N_92_NotABOptBin,0.720475,1.000000,0.259940,0.034708,0.216026
GEN11_SP_N_92_IfSoleJointOtherAccOptBin,0.212860,0.259940,1.000000,-0.018929,0.126108
GEN11_SP_N_92_ByTurnoverOptBin,-0.038274,0.034708,-0.018929,1.000000,-0.480815
GEN11_SP_N_92_ByRPOptBin,0.092571,0.216026,0.126108,-0.480815,1.000000


In [101]:
[col for col in df.columns if (col_name in col) & ('OptBin' in col)]

['GEN11_SP_N_92_ABOtherOptBin',
 'GEN11_SP_N_92_NotABOptBin',
 'GEN11_SP_N_92_IfSoleJointOtherAccOptBin',
 'GEN11_SP_N_92_ByTurnoverOptBin',
 'GEN11_SP_N_92_ByRPOptBin']

In [102]:
COLUMNS_DICT.update({'GEN11_SP_N_92_ByTurnover': 'Char'})
COLUMNS_DICT.update({'GEN11_SP_N_92_ByRP': 'Char'})

#### GEN11_SP_K_80

In [103]:
np.sort(df.GEN11_SP_K_80.unique())

array(['1', '2', '3', '4', '5', '6', 'F', 'F1', 'F2', 'F3', 'F4', 'F5',
       'F6', 'G', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'I', 'I1', 'I3',
       'I4', 'I5', 'I6', 'J', 'J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'S',
       'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'T', 'U', 'U1', 'U2', 'U3',
       'U4', 'U5', 'U6', 'XX'], dtype=object)

In [104]:
col_name='GEN11_SP_K_80'

In [105]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'{T}': 'T', "{'X', 'XX'}": ['X', 'XX'], '{Z}': 'Z'}
)
optb.fit(df_train[col_name], df_train[TARGET_BIN_IisB].values)
df[col_name + 'OptBin'] = optb.transform(df[col_name].values, metric="woe")
df[f'Bin{col_name}'] = optb.transform(df[col_name].values, metric="bins")
dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-excl.jblb'))
df_optb = optb.binning_table.build()
df_optb

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2815141463.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + 'OptBin'] = optb.transform(df[col_name].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2815141463.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name}'] = optb.transform(df[col_name].values, metric="bins")


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"[I4, U4, U3, U2, U1, S6, S5, S4, S3, S2, S1, J...",841,0.060330,837,4,0.004756,1.898526,0.100010,0.010908
1,"[G5, 4, 1]",1737,0.124605,1712,25,0.014393,0.781538,0.053712,0.006548
2,"[3, J3, 2, 6, J1, F1]",1130,0.081062,1108,22,0.019469,0.474266,0.014691,0.001819
3,"[G6, J5, G]",3394,0.243472,3281,113,0.033294,-0.076488,0.001477,0.000185
4,[F],1225,0.087877,1177,48,0.039184,-0.24548,0.005951,0.000742
5,"[J, I]",2029,0.145552,1945,84,0.041400,-0.302803,0.015418,0.001920
6,"[S, J4, U, J6, F5, I6]",1208,0.086657,1152,56,0.046358,-0.4211,0.018804,0.002333
7,{T},41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
8,"{'X', 'XX'}",2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190
9,{Z},0,0.000000,0,0,0.000000,0.0,0.000000,0.000000


##### GEN11_SP_K_80_JSOther

Sole Account: ['S', 'U', 'F', 'G', 'I', 'J',]

Joint Account: [1-6]

In [106]:
col_name: str = 'GEN11_SP_K_80'

In [107]:
SOLE_VALUES = [
    'S',
    'U',
    'F',
    'G',
    'I',
    'J',
]
SOLE_VALUES = '[' + ''.join(SOLE_VALUES) + ']+'
JOINT_VALUES = '[' + ''.join([str(elem) for elem in range(1, 7)]) + ']+'

In [108]:
# patterns to search for
print(JOINT_VALUES)
print(SOLE_VALUES)

[123456]+
[SUFGIJ]+


In [109]:
print('# cases with "Sole" and "Joint" at the same value: ', (df[col_name].str.contains(SOLE_VALUES) & df[col_name].str.contains(JOINT_VALUES)).sum())

# cases with "Sole" and "Joint" at the same value:  8359


In [110]:
df_train[col_name + '_JSOther'] = 'Other'

df_train[col_name + '_JSOther'] = df_train[col_name + '_JSOther'].mask(df_train[col_name].str.contains(SOLE_VALUES) & ~df_train[col_name].str.contains(JOINT_VALUES), 'Sole').values
df_train[col_name + '_JSOther'] = df_train[col_name + '_JSOther'].mask(df_train[col_name].str.contains(JOINT_VALUES) & ~df_train[col_name].str.contains(SOLE_VALUES), 'Joint').values
df_train[col_name + '_JSOther'] = df_train[col_name + '_JSOther'].mask(df_train[col_name].str.contains(SOLE_VALUES) & df_train[col_name].str.contains(JOINT_VALUES), 'Sole&Joint').values
df_train[col_name + '_JSOther'].value_counts(normalize=True)

GEN11_SP_K_80_JSOther
Sole          0.545624
Joint         0.200072
Other         0.170445
Sole&Joint    0.083859
Name: proportion, dtype: float64

In [111]:
df[col_name + '_JSOther'] = 'Other'

df[col_name + '_JSOther'] = df[col_name + '_JSOther'].mask(df[col_name].str.contains(SOLE_VALUES) & ~df[col_name].str.contains(JOINT_VALUES), 'Sole').values
df[col_name + '_JSOther'] = df[col_name + '_JSOther'].mask(df[col_name].str.contains(JOINT_VALUES) & ~df[col_name].str.contains(SOLE_VALUES), 'Joint').values
df[col_name + '_JSOther'] = df[col_name + '_JSOther'].mask(df[col_name].str.contains(SOLE_VALUES) & df[col_name].str.contains(JOINT_VALUES), 'Sole&Joint').values
df[col_name + '_JSOther'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1358001563.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_JSOther'] = 'Other'


GEN11_SP_K_80_JSOther
Sole          0.562510
Other         0.183946
Joint         0.177871
Sole&Joint    0.075673
Name: proportion, dtype: float64

In [112]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip"
)
optb.fit(df_train[col_name + '_JSOther'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_JSOther' + 'OptBin'] = optb.transform(df[col_name + '_JSOther'].values, metric="woe")
df[f'Bin{col_name + '_JSOther'}'] = optb.transform(df[col_name + '_JSOther'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_JSOther'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_JSOther'}-negatives-separately-I-excl.jblb'))
df_JSOther = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2604599280.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_JSOther' + 'OptBin'] = optb.transform(df[col_name + '_JSOther'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2604599280.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_JSOther'}'] = optb.transform(df[col_name + '_JSOther'].values, metric="bins")


In [113]:
df_JSOther

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[Joint],2789,0.200072,2745,44,0.015776,0.688343,0.069598,0.008532
1,[Sole&Joint],1169,0.083859,1150,19,0.016253,0.658075,0.027011,0.003317
2,[Other],2376,0.170445,2297,79,0.033249,-0.075092,0.000996,0.000124
3,[Sole],7606,0.545624,7317,289,0.037996,-0.213474,0.027516,0.003433
4,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
5,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.125120,0.015406


##### GEN11_SP_K_80_NotJS

In [114]:
df_train[col_name + '_NotJS'] = df_train[col_name].copy()

df_train[col_name + '_NotJS'] = df_train[col_name + '_NotJS'].mask(df_train[col_name].str.contains(SOLE_VALUES) & ~df_train[col_name].str.contains(JOINT_VALUES), 'Sole').values
df_train[col_name + '_NotJS'] = df_train[col_name + '_NotJS'].mask(df_train[col_name].str.contains(JOINT_VALUES) & ~df_train[col_name].str.contains(SOLE_VALUES), 'Joint').values
df_train[col_name + '_NotJS'] = df_train[col_name + '_NotJS'].mask(df_train[col_name].str.contains(SOLE_VALUES) & df_train[col_name].str.contains(JOINT_VALUES), 'Sole&Joint').values
df_train[col_name + '_NotJS'].value_counts(normalize=True)

GEN11_SP_K_80_NotJS
Sole          0.545624
Joint         0.200072
XX            0.167504
Sole&Joint    0.083859
T             0.002941
Name: proportion, dtype: float64

In [115]:
df[col_name + '_NotJS'] = df[col_name].copy()

df[col_name + '_NotJS'] = df[col_name + '_NotJS'].mask(df[col_name].str.contains(SOLE_VALUES) & ~df[col_name].str.contains(JOINT_VALUES), 'Sole').values
df[col_name + '_NotJS'] = df[col_name + '_NotJS'].mask(df[col_name].str.contains(JOINT_VALUES) & ~df[col_name].str.contains(SOLE_VALUES), 'Joint').values
df[col_name + '_NotJS'] = df[col_name + '_NotJS'].mask(df[col_name].str.contains(SOLE_VALUES) & df[col_name].str.contains(JOINT_VALUES), 'Sole&Joint').values
df[col_name + '_NotJS'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2660031167.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_NotJS'] = df[col_name].copy()


GEN11_SP_K_80_NotJS
Sole          0.562510
XX            0.180786
Joint         0.177871
Sole&Joint    0.075673
T             0.003159
Name: proportion, dtype: float64

In [116]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'XX': 'XX', 'T': 'T'}
)
optb.fit(df_train[col_name + '_NotJS'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_NotJS' + 'OptBin'] = optb.transform(df[col_name + '_NotJS'].values, metric="woe")
df[f'Bin{col_name + '_NotJS'}'] = optb.transform(df[col_name + '_NotJS'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_NotJS'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_NotJS'}-negatives-separately-I-excl.jblb'))
df_NotJS = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\92129931.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_NotJS' + 'OptBin'] = optb.transform(df[col_name + '_NotJS'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\92129931.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_NotJS'}'] = optb.transform(df[col_name + '_NotJS'].values, metric="bins")


In [117]:
df_NotJS

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[Joint],2789,0.200072,2745,44,0.015776,0.688343,0.069598,0.008532
1,[Sole&Joint],1169,0.083859,1150,19,0.016253,0.658075,0.027011,0.003317
2,[Sole],7606,0.545624,7317,289,0.037996,-0.213474,0.027516,0.003433
3,XX,2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190
4,T,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
5,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.125642,0.015471


In [118]:
df[[col_name + 'OptBin', col_name + '_JSOtherOptBin', col_name + '_NotJSOptBin']].corr()

,GEN11_SP_K_80OptBin,GEN11_SP_K_80_JSOtherOptBin,GEN11_SP_K_80_NotJSOptBin
GEN11_SP_K_80OptBin,1.000000,0.822302,0.821748
GEN11_SP_K_80_JSOtherOptBin,0.822302,1.000000,0.997033
GEN11_SP_K_80_NotJSOptBin,0.821748,0.997033,1.000000


In [119]:
# IV for GEN11_SP_K-based features
print('GEN11_SP_K_80OptBin: ', df_optb.loc['Totals', 'IV'])
print('GEN11_SP_K_80_JSOtherOptBin: ', df_JSOther.loc['Totals', 'IV'])
print('GEN11_SP_K_80_NotJSOptBin: ', df_NotJS.loc['Totals', 'IV'])

GEN11_SP_K_80OptBin:  0.2115791604131389
GEN11_SP_K_80_JSOtherOptBin:  0.12512024488032858
GEN11_SP_K_80_NotJSOptBin:  0.12564163053916422


In [120]:
del df[col_name + '_JSOther']
del df[col_name + '_JSOther' + 'OptBin'], df[f'Bin{col_name + '_JSOther'}']

In [121]:
os.remove(os.path.join(OPT_BIN_PATH, f'{col_name + '_JSOther'}-negatives-separately-I-excl.jblb'))
os.remove(os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_JSOther'}-negatives-separately-I-excl.jblb'))

In [122]:
COLUMNS_DICT.update({'GEN11_SP_K_80_NotJS': 'Char'})
COLUMNS_DICT.update({'GEN11_SP_K_80': 'Char'})

##### GEN11_SP_K_80_TO

In [123]:
df_train[col_name + '_TO'] = df_train[col_name].copy()
for values in ([1, 2], [3, 4], [5, 6]):
    v_comb=''.join([str(v) for v in values])
    df_train[col_name + '_TO'] = df_train[col_name + '_TO'].mask(df_train[col_name].str.contains(r'[{}]+'.format(v_comb)), v_comb).values

df_train[col_name + '_TO'].value_counts(normalize=True)

GEN11_SP_K_80_TO
G     0.235940
XX    0.167504
34    0.159469
J     0.106026
F     0.087877
12    0.071377
S     0.061908
56    0.053085
I     0.039527
U     0.014347
T     0.002941
Name: proportion, dtype: float64

In [124]:
df[col_name + '_TO'] = df[col_name].copy()
for values in ([1, 2], [3, 4], [5, 6]):
    v_comb=''.join([str(v) for v in values])
    df[col_name + '_TO'] = df[col_name + '_TO'].mask(df[col_name].str.contains(r'[{}]+'.format(v_comb)), v_comb).values

df[col_name + '_TO'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\3952446793.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_TO'] = df[col_name].copy()


GEN11_SP_K_80_TO
G     0.254540
XX    0.180786
34    0.142827
F     0.096359
J     0.090918
12    0.069001
S     0.061831
56    0.041716
I     0.036040
U     0.022822
T     0.003159
Name: proportion, dtype: float64

In [125]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'XX': 'XX', 'T': 'T', '[G, F]': ['G', 'F']}
)
optb.fit(df_train[col_name + '_TO'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_TO' + 'OptBin'] = optb.transform(df[col_name + '_TO'].values, metric="woe")
df[f'Bin{col_name + '_TO'}'] = optb.transform(df[col_name + '_TO'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_TO'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_TO'}-negatives-separately-I-excl.jblb'))
df_TO = optb.binning_table.build()

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2466554067.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_TO' + 'OptBin'] = optb.transform(df[col_name + '_TO'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2466554067.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_TO'}'] = optb.transform(df[col_name + '_TO'].values, metric="bins")


In [126]:
df_TO

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[34],2223,0.159469,2192,31,0.013945,0.813579,0.073496,0.008942
1,[12],995,0.071377,980,15,0.015075,0.734499,0.027721,0.003389
2,[56],740,0.053085,723,17,0.022973,0.305193,0.004296,0.000535
3,"[J, I]",2029,0.145552,1945,84,0.041400,-0.302803,0.015418,0.001920
4,"[S, U]",1063,0.076255,1016,47,0.044214,-0.371522,0.012572,0.001563
5,XX,2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190
6,T,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
7,"[G, F]",4514,0.323816,4356,158,0.035002,-0.128289,0.005662,0.000707
8,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.140683,0.017245


In [127]:
df_train[col_name + '_TO'] = df_train[col_name].copy()
for values in ([1, 2, 3, 4], [5, 6]):
    v_comb=''.join([str(v) for v in values])
    df_train[col_name + '_TO'] = df_train[col_name + '_TO'].mask(df_train[col_name].str.contains(r'[{}]+'.format(v_comb)), v_comb).values

df_train[col_name + '_TO'].value_counts(normalize=True)

GEN11_SP_K_80_TO
G       0.235940
1234    0.230846
XX      0.167504
J       0.106026
F       0.087877
S       0.061908
56      0.053085
I       0.039527
U       0.014347
T       0.002941
Name: proportion, dtype: float64

In [128]:
df[col_name + '_TO'] = df[col_name].copy()
for values in ([1, 2, 3, 4], [5, 6]):
    v_comb=''.join([str(v) for v in values])
    df[col_name + '_TO'] = df[col_name + '_TO'].mask(df[col_name].str.contains(r'[{}]+'.format(v_comb)), v_comb).values

df[col_name + '_TO'].value_counts(normalize=True)

GEN11_SP_K_80_TO
G       0.254540
1234    0.211829
XX      0.180786
F       0.096359
J       0.090918
S       0.061831
56      0.041716
I       0.036040
U       0.022822
T       0.003159
Name: proportion, dtype: float64

In [129]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'XX': 'XX', 'T': 'T', '[G, F, S, U]': ['G', 'F', 'S', 'U']}
)
optb.fit(df_train[col_name + '_TO'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_TO' + 'OptBin'] = optb.transform(df[col_name + '_TO'].values, metric="woe")
df[f'Bin{col_name + '_TO'}'] = optb.transform(df[col_name + '_TO'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_TO'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_TO'}-negatives-separately-I-excl.jblb'))
df_TO = optb.binning_table.build()
df_TO

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[1234],3218,0.230846,3172,46,0.014295,0.788473,0.100986,0.012306
1,[56],740,0.053085,723,17,0.022973,0.305193,0.004296,0.000535
2,"[J, I]",2029,0.145552,1945,84,0.041400,-0.302803,0.015418,0.001920
3,XX,2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190
4,T,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
5,"[G, F, S, U]",5577,0.400072,5372,205,0.036758,-0.179058,0.013962,0.001743
6,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.136180,0.016693


##### GEN11_SP_K_80_PR

In [130]:
df_train[col_name + '_PR'] = df_train[col_name].copy()
for values in ([1, 3, 5], [2, 4, 6]):
    v_comb=''.join([str(v) for v in values])
    df_train[col_name + '_PR'] = df_train[col_name + '_PR'].mask(df_train[col_name].str.contains(r'[{}]+'.format(v_comb)), v_comb).values

df_train[col_name + '_PR'].value_counts(normalize=True)

GEN11_SP_K_80_PR
G      0.235940
XX     0.167504
246    0.153085
135    0.130846
J      0.106026
F      0.087877
S      0.061908
I      0.039527
U      0.014347
T      0.002941
Name: proportion, dtype: float64

In [131]:
df[col_name + '_PR'] = df[col_name].copy()
for values in ([1, 3, 5], [2, 4, 6]):
    v_comb=''.join([str(v) for v in values])
    df[col_name + '_PR'] = df[col_name + '_PR'].mask(df[col_name].str.contains(r'[{}]+'.format(v_comb)), v_comb).values

df[col_name + '_PR'].value_counts(normalize=True)

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\3541007424.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_PR'] = df[col_name].copy()


GEN11_SP_K_80_PR
G      0.254540
XX     0.180786
246    0.137912
135    0.115633
F      0.096359
J      0.090918
S      0.061831
I      0.036040
U      0.022822
T      0.003159
Name: proportion, dtype: float64

In [132]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'XX': 'XX', 'T': 'T'}
)
optb.fit(df_train[col_name + '_PR'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_PR' + 'OptBin'] = optb.transform(df[col_name + '_PR'].values, metric="woe")
df[f'Bin{col_name + '_PR'}'] = optb.transform(df[col_name + '_PR'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_PR'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_PR'}-negatives-separately-I-excl.jblb'))
df_PR = optb.binning_table.build()
df_PR

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2230905759.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + '_PR' + 'OptBin'] = optb.transform(df[col_name + '_PR'].values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\2230905759.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Bin{col_name + '_PR'}'] = optb.transform(df[col_name + '_PR'].values, metric="bins")


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[135],1824,0.130846,1798,26,0.014254,0.79133,0.057586,0.007016
1,[246],2134,0.153085,2097,37,0.017338,0.592342,0.041098,0.005063
2,[G],3289,0.235940,3179,110,0.033445,-0.081162,0.001615,0.000202
3,[F],1225,0.087877,1177,48,0.039184,-0.24548,0.005951,0.000742
4,"[J, I]",2029,0.145552,1945,84,0.041400,-0.302803,0.015418,0.001920
5,"[S, U]",1063,0.076255,1016,47,0.044214,-0.371522,0.012572,0.001563
6,XX,2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190
7,T,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
8,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.135758,0.016695


In [133]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'XX': 'XX', 'T': 'T', '[S, F, I]': ['S', 'F', 'I'], '[U, G, J]': ['U', 'G', 'J']}
)
optb.fit(df_train[col_name + '_PR'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_PR' + 'OptBin'] = optb.transform(df[col_name + '_PR'].values, metric="woe")
df[f'Bin{col_name + '_PR'}'] = optb.transform(df[col_name + '_PR'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_PR'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_PR'}-negatives-separately-I-excl.jblb'))
df_PR = optb.binning_table.build()

In [134]:
df_PR

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[135],1824,0.130846,1798,26,0.014254,0.79133,0.057586,0.007016
1,[246],2134,0.153085,2097,37,0.017338,0.592342,0.041098,0.005063
2,XX,2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190
3,T,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
4,"[S, F, I]",2639,0.189311,2531,108,0.040925,-0.290765,0.018383,0.002290
5,"[U, G, J]",4967,0.356313,4786,181,0.036441,-0.17005,0.011167,0.001394
6,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.129752,0.015953


In [135]:
optb = OptimalBinning(
    name=col_name, 
    dtype='categorical', 
    solver="mip",
    special_codes={'XX': 'XX', 'T': 'T', '[S, F, I, U, G, J]': ['S', 'F', 'I', 'U', 'G', 'J']}
)
optb.fit(df_train[col_name + '_PR'], df_train[TARGET_BIN_IisB].values)
df[col_name + '_PR' + 'OptBin'] = optb.transform(df[col_name + '_PR'].values, metric="woe")
df[f'Bin{col_name + '_PR'}'] = optb.transform(df[col_name + '_PR'].values, metric="bins")

dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name + '_PR'}-negatives-separately-I-excl.jblb'))
optbins_params = optb.get_params()
dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name + '_PR'}-negatives-separately-I-excl.jblb'))
df_PR = optb.binning_table.build()
df_PR

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,[135],1824,0.130846,1798,26,0.014254,0.79133,0.057586,0.007016
1,[246],2134,0.153085,2097,37,0.017338,0.592342,0.041098,0.005063
2,XX,2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190
3,T,41,0.002941,41,0,0.000000,0.0,0.000000,0.000000
4,"[S, F, I, U, G, J]",7606,0.545624,7317,289,0.037996,-0.213474,0.027516,0.003433
5,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,13940,1.000000,13509,431,0.030918,,0.127718,0.015702


In [136]:
df[[col for col in df.columns if (col_name in col) & ('OptBin' in col)]].corr()

,GEN11_SP_K_80OptBin,GEN11_SP_K_80_NotJSOptBin,GEN11_SP_K_80_TOOptBin,GEN11_SP_K_80_PROptBin
GEN11_SP_K_80OptBin,1.000000,0.821748,0.799750,0.818617
GEN11_SP_K_80_NotJSOptBin,0.821748,1.000000,0.921943,0.963085
GEN11_SP_K_80_TOOptBin,0.799750,0.921943,1.000000,0.917073
GEN11_SP_K_80_PROptBin,0.818617,0.963085,0.917073,1.000000


In [137]:
print('GEN11_SP_K_80OptBin: ', df_optb.loc['Totals', 'IV'])
print('GEN11_SP_K_80_PROptBin: ', df_PR.loc['Totals', 'IV'])
print('GEN11_SP_K_80_TOOptBin: ', df_TO.loc['Totals', 'IV'])
print('GEN11_SP_K_80_NotJSOptBin: ', df_NotJS.loc['Totals', 'IV'])

GEN11_SP_K_80OptBin:  0.2115791604131389
GEN11_SP_K_80_PROptBin:  0.12771764879805306
GEN11_SP_K_80_TOOptBin:  0.1361795372971974
GEN11_SP_K_80_NotJSOptBin:  0.12564163053916422


In [138]:
df[[col for col in df.columns if (col_name in col) & ('OptBin' in col) & ~('K_80OptBin' in col)]].corr() 

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1245866297.py:1: DeprecationWarning: Bitwise inversion '~' on bool is deprecated. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  df[[col for col in df.columns if (col_name in col) & ('OptBin' in col) & ~('K_80OptBin' in col)]].corr()


,GEN11_SP_K_80_NotJSOptBin,GEN11_SP_K_80_TOOptBin,GEN11_SP_K_80_PROptBin
GEN11_SP_K_80_NotJSOptBin,1.000000,0.921943,0.963085
GEN11_SP_K_80_TOOptBin,0.921943,1.000000,0.917073
GEN11_SP_K_80_PROptBin,0.963085,0.917073,1.000000


In [139]:
for col in ['GEN11_SP_K_80', 'GEN11_SP_K_80_PR', 'GEN11_SP_K_80_NotJS']:
    del df[col + 'OptBin'], df[f'Bin{col}']
    
    os.remove(os.path.join(OPT_BIN_PATH, f'{col}-negatives-separately-I-excl.jblb'))
    os.remove(os.path.join(OPT_BIN_PATH, f'[params]{col}-negatives-separately-I-excl.jblb'))

    if COLUMNS_DICT.get(col) is not None:
        del COLUMNS_DICT[col]

In [140]:
df[[col for col in df.columns if (col_name in col) & ('OptBin' in col) & ~('K_80OptBin' in col)]].corr() 

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\1245866297.py:1: DeprecationWarning: Bitwise inversion '~' on bool is deprecated. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  df[[col for col in df.columns if (col_name in col) & ('OptBin' in col) & ~('K_80OptBin' in col)]].corr()


,GEN11_SP_K_80_TOOptBin
GEN11_SP_K_80_TOOptBin,1.0


In [141]:
COLUMNS_DICT.update({'GEN11_SP_K_80_TO': 'Char'})

### Separate specific values

In [142]:
np.sort(df.SP_B2_20.unique())

array([  -1,    0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
         10,   11,   12,   13,   14,   15,   16,   17,   18,   19,   20,
         21,   22,   23,   24,   25,   26,   27,   28,   29,   30,   31,
         32,   33,   34,   35,   36,   37,   38,   39,   40,   41,   42,
         43,   44,   45,   46,   47,   48,   49,   50,   51,   52,   53,
         54,   55,   56,   57,   58,   59,   60,   61,   62,   63,   64,
         65,   66,   67,   68,   69,   70,   71,   72,   73,   74,   75,
         76,   77,   78,   79,   80,   81,   82,   83,   84,   85,   86,
         87,   88,   89,   90,   91,   92,   93,   94,   95,   96,   97,
         98,   99,  100,  101,  102,  103,  104,  105,  106,  107,  108,
        109,  110,  111,  112,  113,  114,  115,  116,  117,  118,  119,
        120,  121,  122,  123,  124,  125,  126,  127,  128,  129,  130,
        131,  132,  133,  134,  135,  136,  137,  138,  139,  140,  141,
        142,  143,  144,  145,  146,  147,  148,  1

In [143]:
COLUMNS_SP_SPLITS: Dict[str, Any] = {

    'SP_B2_20': dict(
        zip(
            [
                9997,
                9998,
                9999,
            ],
            [
                9997,
                9998,
                9999,
            ]
        )
    ),

    

}

In [144]:
exceptions = []
for col_name in tqdm(COLUMNS_SP_SPLITS.keys()):
    try: 
        unique_vals = df[col_name].unique()
        special_cases: List[Any] = []
        dict_to_replace: Dict[str, float] = dict()
        unique_floats: List[float] = []
        for elem in unique_vals:
            try:
                dict_to_replace[elem] = float(elem)
                unique_floats += [float(elem)]
            except ValueError:
                special_cases += [elem] 

        if len(special_cases) > 0:
            raise ValueError()

        float(elem) 
        optb = OptimalBinning(
            name=col_name, 
            dtype='numerical', 
            solver="mip",
            special_codes=COLUMNS_SP_SPLITS[col_name],
            monotonic_trend='auto_asc_desc', 
            split_digits=0
        )
        optb.fit(df_train[col_name].replace(dict_to_replace).values, df_train[TARGET_BIN_IisB].values)
        df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
        df[f'Bin{col_name}'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="bins")
        dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
        optbins_params = optb.get_params()
        temp = optb.binning_table.build()
        print(temp)
        dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-excl.jblb'))
        temp = optb.binning_table.build()
    except Exception as e:
            exceptions += [(col_name, e)]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.22it/s]

                   Bin  Count  Count (%)  Non-event  Event  Event rate  \
0        (-inf, 38.00)   1107   0.079412       1091     16    0.014453   
1       [38.00, 94.00)   5829   0.418149       5726    103    0.017670   
2         [94.00, inf)   2411   0.172956       2353     58    0.024056   
3                 9997     78   0.005595         67     11    0.141026   
4                 9998   4487   0.321879       4245    242    0.053934   
5                 9999     28   0.002009         27      1    0.035714   
6              Missing      0   0.000000          0      0    0.000000   
Totals                  13940   1.000000      13509    431    0.030918   

             WoE        IV        JS  
0       0.777258  0.033918  0.004136  
1        0.57304  0.105947  0.013065  
2          0.258  0.010219  0.001274  
3      -1.638206  0.033685  0.003795  
4      -0.580444  0.143515  0.017692  
5      -0.149166  0.000048  0.000006  
6            0.0  0.000000  0.000000  
Totals            0.3

In [145]:
print('If any Exception: ', len(exceptions))

If any Exception:  0


### Separate specific neg. values, other negative - as part of numeric, edges as ints

In [146]:
COLUMNS_SP_SPLITS: Dict[str, Any] = {
    'TRD_B_24':  {
        'special_codes':
        dict(
            zip(
                [
                    -999999,
                    -999998,
                    -999997,
                    -999996,
                    -999995,
                ], 
                [
                    -999999,
                    -999998,
                    -999997,
                    -999996,
                    -999995,
                ]
            )
        ),
        'user_splits': [0 , 50,  100],
        'user_splits_fixed': [True, True, True]
    },
    'TRD_B_20': {
        'special_codes': dict(
            zip(
                [
                    -999999,
                    -999998,
                    -999997,
                    -999996,
                    -999995,
                    -100
                ], 
                [
                    -999999,
                    -999998,
                    -999997,
                    -999996,
                    -999995,
                    -100
                ]
            )
        ),
        'user_splits': [0 ,   450, 800],
        'user_splits_fixed': [True, True, True]
    },
    'SP_B2_18': {
        'special_codes': dict(
            zip(
                [
                    9997,
                    9998,
                    9999,
                    -1
                ], 
                [
                    9997,
                    9998,
                    9999,
                    -1
                ]
            )
        )
    },
    'TRD_B_18': {
        'special_codes': dict(
            zip(
                [
                    -999999,
                    -999998,
                    -999997,
                    -999996,
                    -999995,
                    -999994,
                    
                ], 
                [
                    -999999,
                    -999998,
                    -999997,
                    -999996,
                    -999995,
                    -999994,
                ]
            )
        ),
        'user_splits': [0, 100],
        'user_splits_fixed': [True, True]
    },
    'TRD_B_08': {
        'special_codes': 
            dict(
                zip(
                    [
                        -999999,
                        -999998,
                        -999997,
                        -999996,
                        -999995,
                        -999994,
                    ],
                    [
                        -999999,
                        -999998,
                        -999997,
                        -999996,
                        -999995,
                        -999994,
                    ]
                )
        ),
    'user_splits': [0, 100],
    'user_splits_fixed': [True, True]
    }, 

    'ND_ECC_04': {
        'special_codes': dict(
            zip(
                [
                    -1,
                    997,
                    998,
                    999,
                ],
                [
                    -1,
                    997,
                    998,
                    999,
                ]
            )
        ),
    'user_splits': [5],
    'user_splits_fixed': [True] 
    }
}

In [147]:
np.sort(df.ND_ECC_04.unique())

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  84,  85,  86,  89,  92,  93,  97,  99, 101, 105, 107,
       108, 111, 118, 121, 123, 124, 126, 131, 146, 148, 173, 187, 195,
       202, 209, 263, 267, 289, 998, 999], dtype=int64)

In [148]:
exceptions = []
for col_name in tqdm(COLUMNS_SP_SPLITS.keys()):
    try: 
        unique_vals = df[col_name].unique()
        
        dict_to_replace: Dict[str, float] = dict()
        unique_floats: List[float] = []
        for elem in unique_vals:
            try:
                dict_to_replace[elem] = float(elem)
                unique_floats += [float(elem)]
            except ValueError:
                pass


        optb = OptimalBinning(
            name=col_name, 
            dtype='numerical', 
            solver="mip",
            special_codes=COLUMNS_SP_SPLITS[col_name]['special_codes'],
            monotonic_trend='auto_asc_desc',
            user_splits=COLUMNS_SP_SPLITS[col_name].get('user_splits'),
            user_splits_fixed=COLUMNS_SP_SPLITS[col_name].get('user_splits_fixed'), 
            min_n_bins=len(COLUMNS_SP_SPLITS[col_name]['special_codes']) + \
                sum(COLUMNS_SP_SPLITS[col_name].get('user_splits_fixed') if COLUMNS_SP_SPLITS[col_name].get('user_splits_fixed') is not None else [0]),
            split_digits=0
        )
        optb.fit(df_train[col_name].replace(dict_to_replace).values, df_train[TARGET_BIN_IisB].values)
        df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
        df[f'Bin{col_name}'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="bins")
        dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
        optbins_params = optb.get_params()
        print(optbins_params)
        dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-excl.jblb'))
        temp = optb.binning_table.build()
        print(temp)
    except Exception as e:
            exceptions += [(col_name, e)]

 17%|██████████████                                                                      | 1/6 [00:02<00:10,  2.01s/it]

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': 8, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'TRD_B_24', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-999999: -999999, -999998: -999998, -999997: -999997, -999996: -999996, -999995: -999995}, 'split_digits': 0, 'time_limit': 100, 'user_splits': [0, 50, 100], 'user_splits_fixed': array([ True,  True,  True]), 'verbose': False}
                    Bin  Count  Count (%)  Non-event  Event  Event rate  \
0          (-inf, 0.00)   4722   0.338737       4672     50    0.010589   

 33%|████████████████████████████                                                        | 2/6 [00:07<00:16,  4.07s/it]

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': 9, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'TRD_B_20', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-999999: -999999, -999998: -999998, -999997: -999997, -999996: -999996, -999995: -999995, -100: -100}, 'split_digits': 0, 'time_limit': 100, 'user_splits': [0, 450, 800], 'user_splits_fixed': array([ True,  True,  True]), 'verbose': False}
                     Bin  Count  Count (%)  Non-event  Event  Event rate  \
0           (-inf, 0.00)   3656   0.262267       3612     44

 50%|██████████████████████████████████████████                                          | 3/6 [00:08<00:07,  2.62s/it]

...

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': 8, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'TRD_B_18', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-999999: -999999, -999998: -999998, -999997: -999997, -999996: -999996, -999995: -999995, -999994: -999994}, 'split_digits': 0, 'time_limit': 100, 'user_splits': [0, 100], 'user_splits_fixed': array([ True,  True]), 'verbose': False}
                   Bin  Count  Count (%)  Non-event  Event  Event rate  \
0         (-inf, 0.00)   5431   0.389598       5355     76    0.0139

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [00:19<00:04,  4.65s/it]

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': 8, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'TRD_B_08', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-999999: -999999, -999998: -999998, -999997: -999997, -999996: -999996, -999995: -999995, -999994: -999994}, 'split_digits': 0, 'time_limit': 100, 'user_splits': [0, 100], 'user_splits_fixed': array([ True,  True]), 'verbose': False}
                   Bin  Count  Count (%)  Non-event  Event  Event rate  \
0         (-inf, 0.00)   5201   0.373099       5109     92    0.0176

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.34s/it]

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': 5, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'ND_ECC_04', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-1: -1, 997: 997, 998: 998, 999: 999}, 'split_digits': 0, 'time_limit': 100, 'user_splits': [5], 'user_splits_fixed': array([ True]), 'verbose': False}
                 Bin  Count  Count (%)  Non-event  Event  Event rate  \
0       (-inf, 5.00)    822   0.058967        729     93    0.113139   
1        [5.00, inf)    532   0.038164        518     14    0.026316   
2      

In [149]:
print('If any Exception: ', len(exceptions))

If any Exception:  0


## WoE per bin

In [242]:
COLS_INTER = [
    'E4_Q_17',
    'TRD_A_20',
    # 'ND_ERL_01',
    'SP_B2_20',
    'TRD_B_24',
    'TRD_B_20',
    'SP_B2_18',
    'TRD_B_18',
    'TRD_B_08',
    'ND_ECC_04', 
    'GEN11_SP_N_92_ABOther',
    'GEN11_SP_N_92_NotAB',
    'GEN11_SP_N_92_IfSoleJointOtherAcc',
    'GEN11_SP_N_92_ByTurnover',
    'GEN11_SP_N_92_ByRP', 
    'GEN11_SP_K_80_TO',
]

In [243]:
def find_max_of_negative_bins(x):
    v: Union[float, np.nan] = np.nan
    try: 
        v = float(x)
        return v
    except:
        return np.nan

In [244]:
#  extract intervals
warnings.filterwarnings("ignore", category=FutureWarning)
df_intervals = pd.DataFrame()
for col_name in COLUMNS_DICT.keys():
    optb = load(os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-excl.jblb'))
    temp = optb.binning_table.build(add_totals=False)
    temp['Feature'] = col_name
    temp.Bin = temp.Bin.astype(str)
    temp = temp[~((temp.Bin == 'Missing') & (temp.Count == 0))]
    temp = temp[~((temp.Bin == 'Special') & (temp.Count == 0))]
    temp = temp[(temp.Bin != '')]
    temp = temp.reset_index(drop=True)
    df_intervals = pd.concat((df_intervals, temp))

In [245]:
df_intervals.head()

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature
0,"(-inf, 684.50)",1327,0.095194,1145,182,0.137151,-1.605850,0.541999,0.061296,OPTIN_11SCN_ML
1,"[684.50, 760.50)",1100,0.078910,1001,99,0.090000,-1.131368,0.176040,0.020902,OPTIN_11SCN_ML
2,"[760.50, 814.50)",845,0.060617,796,49,0.057988,-0.657224,0.035993,0.004420,OPTIN_11SCN_ML
3,"[814.50, 859.50)",699,0.050143,676,23,0.032904,-0.064304,0.000214,0.000027,OPTIN_11SCN_ML
4,"[859.50, 955.50)",1661,0.119154,1627,34,0.020470,0.423129,0.017582,0.002181,OPTIN_11SCN_ML


In [246]:
df_intervals[df_intervals.Feature == 'EA5_S_01']

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature
0,"(-inf, 28.00)",836,0.059971,784,52,0.062201,-0.731838,0.045824,0.005603,EA5_S_01
1,"[28.00, 34.00)",1903,0.136514,1795,108,0.056752,-0.634374,0.074669,0.009180,EA5_S_01
2,"[34.00, 36.00)",831,0.059613,799,32,0.038508,-0.227378,0.003433,0.000428,EA5_S_01
3,"[36.00, 38.00)",861,0.061765,829,32,0.037166,-0.190519,0.002454,0.000306,EA5_S_01
4,"[38.00, 42.00)",1727,0.123888,1664,63,0.036479,-0.171158,0.003936,0.000491,EA5_S_01
5,"[42.00, 44.00)",793,0.056887,771,22,0.027743,0.111643,0.000673,0.000084,EA5_S_01
6,"[44.00, 52.00)",2977,0.213558,2910,67,0.022506,0.326212,0.019560,0.002434,EA5_S_01
7,"[52.00, 54.00)",751,0.053874,738,13,0.017310,0.593991,0.014534,0.001790,EA5_S_01
8,"[54.00, inf)",3261,0.233931,3219,42,0.012879,0.894153,0.125931,0.015237,EA5_S_01


In [247]:
df_intervals_st = df_intervals[~df_intervals.Feature.isin(COLS_INTER)].reset_index(drop=True).copy()
df_intervals_mod = df_intervals[df_intervals.Feature.isin(COLS_INTER)].reset_index(drop=True).copy()

In [248]:
temp = df_intervals_st.groupby('Feature').apply(lambda y: y.Bin.apply(lambda x: float(find_max_of_negative_bins(x))).astype(float).max()).reset_index(name='min_val')
df_intervals_st = pd.merge(df_intervals_st, temp, on='Feature', how='left')
df_intervals_st['Bin'] = df_intervals_st.apply(lambda x: x.Bin.replace('(-inf', '(' + str(x.min_val)) if not np.isnan(x.min_val) else x.Bin, axis=1)
del df_intervals_st['min_val']
df_intervals_st

C:\Users\hdychko\AppData\Local\Temp\ipykernel_32672\4025125238.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = df_intervals_st.groupby('Feature').apply(lambda y: y.Bin.apply(lambda x: float(find_max_of_negative_bins(x))).astype(float).max()).reset_index(name='min_val')


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature
0,"(-997.0, 684.50)",1327,0.095194,1145,182,0.137151,-1.605850,0.541999,0.061296,OPTIN_11SCN_ML
1,"[684.50, 760.50)",1100,0.078910,1001,99,0.090000,-1.131368,0.176040,0.020902,OPTIN_11SCN_ML
2,"[760.50, 814.50)",845,0.060617,796,49,0.057988,-0.657224,0.035993,0.004420,OPTIN_11SCN_ML
3,"[814.50, 859.50)",699,0.050143,676,23,0.032904,-0.064304,0.000214,0.000027,OPTIN_11SCN_ML
4,"[859.50, 955.50)",1661,0.119154,1627,34,0.020470,0.423129,0.017582,0.002181,OPTIN_11SCN_ML
...,...,...,...,...,...,...,...,...,...,...
15996,['N'],13029,0.934648,12641,388,0.029780,0.038692,0.001374,0.000172,EA4_Q_06
15997,['Y'],911,0.065352,868,43,0.047201,-0.440012,0.015627,0.001938,EA4_Q_06
15998,"(-1.0, inf)",3330,0.238881,3179,151,0.045345,-0.397961,0.045775,0.005684,VM03_SP_VM2_26
15999,-2.0,10569,0.758178,10289,280,0.026493,0.159038,0.017810,0.002224,VM03_SP_VM2_26


In [249]:
df_intervals_mod['Bin_init'] = df_intervals_mod['Bin'].copy()

#### E4_Q_17

In [250]:
df_intervals_mod[df_intervals_mod.Feature == 'E4_Q_17']

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
13,[6],2744,0.196844,2648,96,0.034985,-0.127792,0.003415,0.000427,E4_Q_17,[6]
14,2,8286,0.594405,8180,106,0.012793,0.901005,0.323986,0.039182,E4_Q_17,2
15,"(1, 5)",243,0.017432,229,14,0.057613,-0.650339,0.010100,0.001241,E4_Q_17,"(1, 5)"
16,"(3, 4, 7, 8)",2667,0.191320,2452,215,0.080615,-1.010982,0.320816,0.038477,E4_Q_17,"(3, 4, 7, 8)"


In [251]:
df_intervals_mod.loc[df_intervals_mod.Feature == 'E4_Q_17', 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == 'E4_Q_17'].Bin.replace(
    {
        '[6]': '{6}',
        '(1, 5)': '{1, 5}',
        '(3, 4, 7, 8)': '{3, 4, 7, 8}',
        '2': '{2}'
    }
).values

In [252]:
df_intervals_mod[df_intervals_mod.Feature == 'E4_Q_17']

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
13,{6},2744,0.196844,2648,96,0.034985,-0.127792,0.003415,0.000427,E4_Q_17,[6]
14,{2},8286,0.594405,8180,106,0.012793,0.901005,0.323986,0.039182,E4_Q_17,2
15,"{1, 5}",243,0.017432,229,14,0.057613,-0.650339,0.010100,0.001241,E4_Q_17,"(1, 5)"
16,"{3, 4, 7, 8}",2667,0.191320,2452,215,0.080615,-1.010982,0.320816,0.038477,E4_Q_17,"(3, 4, 7, 8)"


#### TRD_A_20

In [253]:
col_name = 'TRD_A_20'

In [254]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
0,['1+'],465,0.033357,457,8,0.017204,0.600239,0.009164,0.001129,TRD_A_20,['1+']
1,"{-3, 0}",5977,0.428766,5932,45,0.007529,1.436451,0.480789,0.055412,TRD_A_20,"{-3, 0}"
2,-2,7457,0.534935,7079,378,0.050691,-0.515010,0.181803,0.022478,TRD_A_20,-2
3,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_A_20,-1


In [255]:
df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.values

array(["['1+']", '{-3, 0}', '-2', '-1'], dtype=object)

In [256]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.replace(
    {
        "['1+']": '[1, inf)',
    }
).values

In [257]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
0,"[1, inf)",465,0.033357,457,8,0.017204,0.600239,0.009164,0.001129,TRD_A_20,['1+']
1,"{-3, 0}",5977,0.428766,5932,45,0.007529,1.436451,0.480789,0.055412,TRD_A_20,"{-3, 0}"
2,-2,7457,0.534935,7079,378,0.050691,-0.515010,0.181803,0.022478,TRD_A_20,-2
3,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_A_20,-1


In [258]:
COLS_MAN_CORRECT

{'TRD_A_20': [{'Bin': '[1+]', 'WoE': 0}]}

In [259]:
# change woe
df_intervals_mod.loc[(df_intervals_mod.Feature == col_name) & (df_intervals_mod.Bin == '[1, inf)'), 'WoE'] = 0

In [260]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
0,"[1, inf)",465,0.033357,457,8,0.017204,0.000000,0.009164,0.001129,TRD_A_20,['1+']
1,"{-3, 0}",5977,0.428766,5932,45,0.007529,1.436451,0.480789,0.055412,TRD_A_20,"{-3, 0}"
2,-2,7457,0.534935,7079,378,0.050691,-0.515010,0.181803,0.022478,TRD_A_20,-2
3,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_A_20,-1


#### SP_B2_20


In [261]:
col_name = 'SP_B2_20'

In [262]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
53,"(-inf, 38.00)",1107,0.079412,1091,16,0.014453,0.777258,0.033918,0.004136,SP_B2_20,"(-inf, 38.00)"
54,"[38.00, 94.00)",5829,0.418149,5726,103,0.017670,0.573040,0.105947,0.013065,SP_B2_20,"[38.00, 94.00)"
55,"[94.00, inf)",2411,0.172956,2353,58,0.024056,0.258000,0.010219,0.001274,SP_B2_20,"[94.00, inf)"
56,9997,78,0.005595,67,11,0.141026,-1.638206,0.033685,0.003795,SP_B2_20,9997
57,9998,4487,0.321879,4245,242,0.053934,-0.580444,0.143515,0.017692,SP_B2_20,9998
58,9999,28,0.002009,27,1,0.035714,-0.149166,0.000048,0.000006,SP_B2_20,9999


In [263]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('(-inf', '[0')

In [264]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('inf)', '9996]')

In [265]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
53,"[0, 38.00)",1107,0.079412,1091,16,0.014453,0.777258,0.033918,0.004136,SP_B2_20,"(-inf, 38.00)"
54,"[38.00, 94.00)",5829,0.418149,5726,103,0.017670,0.573040,0.105947,0.013065,SP_B2_20,"[38.00, 94.00)"
55,"[94.00, 9996]",2411,0.172956,2353,58,0.024056,0.258000,0.010219,0.001274,SP_B2_20,"[94.00, inf)"
56,9997,78,0.005595,67,11,0.141026,-1.638206,0.033685,0.003795,SP_B2_20,9997
57,9998,4487,0.321879,4245,242,0.053934,-0.580444,0.143515,0.017692,SP_B2_20,9998
58,9999,28,0.002009,27,1,0.035714,-0.149166,0.000048,0.000006,SP_B2_20,9999


#### TRD_B_24

In [266]:
col_name = 'TRD_B_24'

In [267]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
4,"(-inf, 0.00)",4722,0.338737,4672,50,0.010589,1.092316,0.251052,0.029909,TRD_B_24,"(-inf, 0.00)"
5,"[0.00, 50.00)",3974,0.285079,3861,113,0.028435,0.086290,0.002039,0.000255,TRD_B_24,"[0.00, 50.00)"
6,"[50.00, 100.00)",1148,0.082353,1127,21,0.018293,0.537789,0.018662,0.002305,TRD_B_24,"[50.00, 100.00)"
7,"[100.00, inf)",1563,0.112123,1541,22,0.014075,0.804141,0.050683,0.006170,TRD_B_24,"[100.00, inf)"
8,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_24,-999999
9,-999998,2094,0.150215,1929,165,0.078797,-0.986192,0.236722,0.028447,TRD_B_24,-999998
10,-999997,398,0.028551,338,60,0.150754,-1.716302,0.195986,0.021875,TRD_B_24,-999997
11,-999996,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_24,-999996
12,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_24,-999995


In [268]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('(-inf', '[-100')

In [269]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
4,"[-100, 0.00)",4722,0.338737,4672,50,0.010589,1.092316,0.251052,0.029909,TRD_B_24,"(-inf, 0.00)"
5,"[0.00, 50.00)",3974,0.285079,3861,113,0.028435,0.086290,0.002039,0.000255,TRD_B_24,"[0.00, 50.00)"
6,"[50.00, 100.00)",1148,0.082353,1127,21,0.018293,0.537789,0.018662,0.002305,TRD_B_24,"[50.00, 100.00)"
7,"[100.00, inf)",1563,0.112123,1541,22,0.014075,0.804141,0.050683,0.006170,TRD_B_24,"[100.00, inf)"
8,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_24,-999999
9,-999998,2094,0.150215,1929,165,0.078797,-0.986192,0.236722,0.028447,TRD_B_24,-999998
10,-999997,398,0.028551,338,60,0.150754,-1.716302,0.195986,0.021875,TRD_B_24,-999997
11,-999996,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_24,-999996
12,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_24,-999995


#### TRD_B_20

In [270]:
col_name = 'TRD_B_20'

In [271]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
17,"(-inf, 0.00)",3656,0.262267,3612,44,0.012035,0.962824,0.159144,0.019159,TRD_B_20,"(-inf, 0.00)"
18,"[0.00, 450.00)",4406,0.316069,4340,66,0.014980,0.740972,0.124583,0.015226,TRD_B_20,"[0.00, 450.00)"
19,"[450.00, 800.00)",358,0.025681,352,6,0.016760,0.626868,0.007607,0.000936,TRD_B_20,"[450.00, 800.00)"
20,"[800.00, inf)",793,0.056887,767,26,0.032787,-0.060613,0.000215,0.000027,TRD_B_20,"[800.00, inf)"
21,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_20,-999999
22,-999998,1904,0.136585,1747,157,0.082458,-1.035594,0.243311,0.029124,TRD_B_20,-999998
23,-999997,398,0.028551,338,60,0.150754,-1.716302,0.195986,0.021875,TRD_B_20,-999997
24,-999996,1536,0.110187,1474,62,0.040365,-0.276403,0.009602,0.001196,TRD_B_20,-999996
25,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_20,-999995
26,-100,848,0.060832,838,10,0.011792,0.983430,0.038187,0.004590,TRD_B_20,-100


In [272]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('(-inf', '(-100')

In [273]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
17,"(-100, 0.00)",3656,0.262267,3612,44,0.012035,0.962824,0.159144,0.019159,TRD_B_20,"(-inf, 0.00)"
18,"[0.00, 450.00)",4406,0.316069,4340,66,0.014980,0.740972,0.124583,0.015226,TRD_B_20,"[0.00, 450.00)"
19,"[450.00, 800.00)",358,0.025681,352,6,0.016760,0.626868,0.007607,0.000936,TRD_B_20,"[450.00, 800.00)"
20,"[800.00, inf)",793,0.056887,767,26,0.032787,-0.060613,0.000215,0.000027,TRD_B_20,"[800.00, inf)"
21,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_20,-999999
22,-999998,1904,0.136585,1747,157,0.082458,-1.035594,0.243311,0.029124,TRD_B_20,-999998
23,-999997,398,0.028551,338,60,0.150754,-1.716302,0.195986,0.021875,TRD_B_20,-999997
24,-999996,1536,0.110187,1474,62,0.040365,-0.276403,0.009602,0.001196,TRD_B_20,-999996
25,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_20,-999995
26,-100,848,0.060832,838,10,0.011792,0.983430,0.038187,0.004590,TRD_B_20,-100


#### SP_B2_18

In [274]:
col_name = 'SP_B2_18'

In [275]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
27,"(-inf, 82.00)",3397,0.243687,3361,36,0.010598,1.091472,0.180388,0.021492,SP_B2_18,"(-inf, 82.00)"
28,"[82.00, 92.00)",1025,0.073529,1014,11,0.010732,1.078760,0.053441,0.006374,SP_B2_18,"[82.00, 92.00)"
29,"[92.00, 156.00)",2436,0.174749,2397,39,0.016010,0.673408,0.058553,0.007184,SP_B2_18,"[92.00, 156.00)"
30,"[156.00, inf)",698,0.050072,684,14,0.020057,0.443897,0.008057,0.000999,SP_B2_18,"[156.00, inf)"
31,9997,576,0.041320,520,56,0.097222,-1.216526,0.111236,0.013106,SP_B2_18,9997
32,9998,5066,0.363415,4880,186,0.036715,-0.177849,0.012505,0.001561,SP_B2_18,9998
33,9999,701,0.050287,612,89,0.126961,-1.516907,0.244515,0.027935,SP_B2_18,9999
34,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,SP_B2_18,-1


In [276]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('(-inf', '[0')

In [277]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('inf)', '9996]')

In [278]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
27,"[0, 82.00)",3397,0.243687,3361,36,0.010598,1.091472,0.180388,0.021492,SP_B2_18,"(-inf, 82.00)"
28,"[82.00, 92.00)",1025,0.073529,1014,11,0.010732,1.078760,0.053441,0.006374,SP_B2_18,"[82.00, 92.00)"
29,"[92.00, 156.00)",2436,0.174749,2397,39,0.016010,0.673408,0.058553,0.007184,SP_B2_18,"[92.00, 156.00)"
30,"[156.00, 9996]",698,0.050072,684,14,0.020057,0.443897,0.008057,0.000999,SP_B2_18,"[156.00, inf)"
31,9997,576,0.041320,520,56,0.097222,-1.216526,0.111236,0.013106,SP_B2_18,9997
32,9998,5066,0.363415,4880,186,0.036715,-0.177849,0.012505,0.001561,SP_B2_18,9998
33,9999,701,0.050287,612,89,0.126961,-1.516907,0.244515,0.027935,SP_B2_18,9999
34,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,SP_B2_18,-1


#### TRD_B_18

In [279]:
col_name = 'TRD_B_18'

In [280]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
35,"(-inf, 0.00)",5431,0.389598,5355,76,0.013994,0.810049,0.178266,0.021693,TRD_B_18,"(-inf, 0.00)"
36,"[0.00, 100.00)",4306,0.308895,4188,118,0.027404,0.124291,0.004504,0.000563,TRD_B_18,"[0.00, 100.00)"
37,"[100.00, inf)",1899,0.136227,1859,40,0.021064,0.393911,0.017649,0.002192,TRD_B_18,"[100.00, inf)"
38,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_18,-999999
39,-999998,856,0.061406,774,82,0.095794,-1.200151,0.159572,0.018829,TRD_B_18,-999998
40,-999997,398,0.028551,338,60,0.150754,-1.716302,0.195986,0.021875,TRD_B_18,-999997
41,-999996,1009,0.072382,954,55,0.054509,-0.591673,0.033720,0.004155,TRD_B_18,-999996
42,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_18,-999995
43,-999994,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_18,-999994


In [281]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('(-inf', '[-100')

In [282]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
35,"[-100, 0.00)",5431,0.389598,5355,76,0.013994,0.810049,0.178266,0.021693,TRD_B_18,"(-inf, 0.00)"
36,"[0.00, 100.00)",4306,0.308895,4188,118,0.027404,0.124291,0.004504,0.000563,TRD_B_18,"[0.00, 100.00)"
37,"[100.00, inf)",1899,0.136227,1859,40,0.021064,0.393911,0.017649,0.002192,TRD_B_18,"[100.00, inf)"
38,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_18,-999999
39,-999998,856,0.061406,774,82,0.095794,-1.200151,0.159572,0.018829,TRD_B_18,-999998
40,-999997,398,0.028551,338,60,0.150754,-1.716302,0.195986,0.021875,TRD_B_18,-999997
41,-999996,1009,0.072382,954,55,0.054509,-0.591673,0.033720,0.004155,TRD_B_18,-999996
42,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_18,-999995
43,-999994,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_18,-999994


#### TRD_B_08

In [283]:
col_name = 'TRD_B_08'

In [284]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
44,"(-inf, 0.00)",5201,0.373099,5109,92,0.017689,0.571967,0.094223,0.011620,TRD_B_08,"(-inf, 0.00)"
45,"[0.00, 100.00)",2931,0.210258,2841,90,0.030706,0.007098,0.000011,0.000001,TRD_B_08,"[0.00, 100.00)"
46,"[100.00, inf)",3452,0.247633,3360,92,0.026651,0.152904,0.005392,0.000673,TRD_B_08,"[100.00, inf)"
47,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999999
48,-999998,568,0.040746,512,56,0.098592,-1.232030,0.113383,0.013339,TRD_B_08,-999998
49,-999997,18,0.001291,18,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999997
50,-999996,1729,0.124032,1628,101,0.058415,-0.665016,0.075696,0.009291,TRD_B_08,-999996
51,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999995
52,-999994,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999994


In [285]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('(-inf', '[-100')

In [286]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
44,"[-100, 0.00)",5201,0.373099,5109,92,0.017689,0.571967,0.094223,0.011620,TRD_B_08,"(-inf, 0.00)"
45,"[0.00, 100.00)",2931,0.210258,2841,90,0.030706,0.007098,0.000011,0.000001,TRD_B_08,"[0.00, 100.00)"
46,"[100.00, inf)",3452,0.247633,3360,92,0.026651,0.152904,0.005392,0.000673,TRD_B_08,"[100.00, inf)"
47,-999999,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999999
48,-999998,568,0.040746,512,56,0.098592,-1.232030,0.113383,0.013339,TRD_B_08,-999998
49,-999997,18,0.001291,18,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999997
50,-999996,1729,0.124032,1628,101,0.058415,-0.665016,0.075696,0.009291,TRD_B_08,-999996
51,-999995,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999995
52,-999994,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,TRD_B_08,-999994


#### ND_ECC_04

In [287]:
col_name = 'ND_ECC_04'

In [288]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
59,"(-inf, 5.00)",822,0.058967,729,93,0.113139,-1.385929,0.224262,0.025985,ND_ECC_04,"(-inf, 5.00)"
60,"[5.00, inf)",532,0.038164,518,14,0.026316,0.165915,0.000973,0.000121,ND_ECC_04,"[5.00, inf)"
61,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,ND_ECC_04,-1
62,997,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,ND_ECC_04,997
63,998,1202,0.086227,1131,71,0.059068,-0.676826,0.054830,0.006726,ND_ECC_04,998
64,999,11343,0.813702,11090,253,0.022305,0.335406,0.078461,0.009762,ND_ECC_04,999


In [289]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('(-inf', '[0')

In [290]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace('inf)', '996]')

In [291]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
59,"[0, 5.00)",822,0.058967,729,93,0.113139,-1.385929,0.224262,0.025985,ND_ECC_04,"(-inf, 5.00)"
60,"[5.00, 996]",532,0.038164,518,14,0.026316,0.165915,0.000973,0.000121,ND_ECC_04,"[5.00, inf)"
61,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,ND_ECC_04,-1
62,997,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,ND_ECC_04,997
63,998,1202,0.086227,1131,71,0.059068,-0.676826,0.054830,0.006726,ND_ECC_04,998
64,999,11343,0.813702,11090,253,0.022305,0.335406,0.078461,0.009762,ND_ECC_04,999


### GEN11_SP_N_92_ABOther

In [292]:
col_name = 'GEN11_SP_N_92_ABOther'

In [293]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
65,['B'],8452,0.606313,8342,110,0.013015,0.883575,0.320114,0.038761,GEN11_SP_N_92_ABOther,['B']
66,['Other'],4405,0.315997,4167,238,0.054030,-0.582322,0.141937,0.017496,GEN11_SP_N_92_ABOther,['Other']
67,['A'],1083,0.077690,1000,83,0.076639,-0.956089,0.113345,0.013652,GEN11_SP_N_92_ABOther,['A']


In [294]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.replace(
    {
        "['B']": "{'B'}",
        "['Other']": "{'Other'}",
        "['A']": "{'A'}",
    }
)

In [295]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
65,{'B'},8452,0.606313,8342,110,0.013015,0.883575,0.320114,0.038761,GEN11_SP_N_92_ABOther,['B']
66,{'Other'},4405,0.315997,4167,238,0.054030,-0.582322,0.141937,0.017496,GEN11_SP_N_92_ABOther,['Other']
67,{'A'},1083,0.077690,1000,83,0.076639,-0.956089,0.113345,0.013652,GEN11_SP_N_92_ABOther,['A']


### GEN11_SP_N_92_NotAB

In [296]:
col_name = 'GEN11_SP_N_92_NotAB'

In [297]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
68,['F' 'G'],1850,0.132712,1763,87,0.047027,-0.436139,0.031119,0.003859,GEN11_SP_N_92_NotAB,['F' 'G']
69,['P'],1063,0.076255,1006,57,0.053622,-0.574317,0.033185,0.004092,GEN11_SP_N_92_NotAB,['P']
70,['S' 'I'],707,0.050717,663,44,0.062235,-0.732418,0.038825,0.004748,GEN11_SP_N_92_NotAB,['S' 'I']
71,['J' 'U'],715,0.051291,665,50,0.069930,-0.857239,0.057249,0.006945,GEN11_SP_N_92_NotAB,['J' 'U']
72,A,1083,0.077690,1000,83,0.076639,-0.956089,0.113345,0.013652,GEN11_SP_N_92_NotAB,A
73,B,8452,0.606313,8342,110,0.013015,0.883575,0.320114,0.038761,GEN11_SP_N_92_NotAB,B
74,T,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,GEN11_SP_N_92_NotAB,T
75,M,29,0.002080,29,0,0.000000,0.000000,0.000000,0.000000,GEN11_SP_N_92_NotAB,M


In [298]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    '[', ''
)
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    ']', ''
)

In [299]:
df_intervals_mod.loc[(df_intervals_mod.Feature == col_name) & (df_intervals_mod.Bin.isin(['A', 'B', 'T', 'M'])), 'Bin'] = '\'' + df_intervals_mod[(df_intervals_mod.Feature == col_name) & (df_intervals_mod.Bin.isin(['A', 'B', 'T', 'M']))].Bin + '\''

In [300]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = '{' + df_intervals_mod[df_intervals_mod.Feature == col_name].Bin + '}'

In [301]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
68,{'F' 'G'},1850,0.132712,1763,87,0.047027,-0.436139,0.031119,0.003859,GEN11_SP_N_92_NotAB,['F' 'G']
69,{'P'},1063,0.076255,1006,57,0.053622,-0.574317,0.033185,0.004092,GEN11_SP_N_92_NotAB,['P']
70,{'S' 'I'},707,0.050717,663,44,0.062235,-0.732418,0.038825,0.004748,GEN11_SP_N_92_NotAB,['S' 'I']
71,{'J' 'U'},715,0.051291,665,50,0.069930,-0.857239,0.057249,0.006945,GEN11_SP_N_92_NotAB,['J' 'U']
72,{'A'},1083,0.077690,1000,83,0.076639,-0.956089,0.113345,0.013652,GEN11_SP_N_92_NotAB,A
73,{'B'},8452,0.606313,8342,110,0.013015,0.883575,0.320114,0.038761,GEN11_SP_N_92_NotAB,B
74,{'T'},41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,GEN11_SP_N_92_NotAB,T
75,{'M'},29,0.002080,29,0,0.000000,0.000000,0.000000,0.000000,GEN11_SP_N_92_NotAB,M


### GEN11_SP_N_92_IfSoleJointOtherAcc

In [302]:
col_name = 'GEN11_SP_N_92_IfSoleJointOtherAcc'

In [303]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
76,['Joint'],1053,0.075538,1038,15,0.014245,0.791998,0.033292,0.004056,GEN11_SP_N_92_IfSoleJointOtherAcc,['Joint']
77,['Other'],5719,0.410258,5550,169,0.029551,0.046651,0.000874,0.000109,GEN11_SP_N_92_IfSoleJointOtherAcc,['Other']
78,['Sole'],7168,0.514204,6921,247,0.034459,-0.112076,0.006810,0.000851,GEN11_SP_N_92_IfSoleJointOtherAcc,['Sole']


In [304]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    '[', ''
)
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    ']', ''
)

In [305]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = '{' + df_intervals_mod[df_intervals_mod.Feature == col_name].Bin + '}'

In [306]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
76,{'Joint'},1053,0.075538,1038,15,0.014245,0.791998,0.033292,0.004056,GEN11_SP_N_92_IfSoleJointOtherAcc,['Joint']
77,{'Other'},5719,0.410258,5550,169,0.029551,0.046651,0.000874,0.000109,GEN11_SP_N_92_IfSoleJointOtherAcc,['Other']
78,{'Sole'},7168,0.514204,6921,247,0.034459,-0.112076,0.006810,0.000851,GEN11_SP_N_92_IfSoleJointOtherAcc,['Sole']


### GEN11_SP_N_92_ByTurnover

In [307]:
col_name = 'GEN11_SP_N_92_ByTurnover'

In [308]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
79,['Fluctuating'],1710,0.122669,1663,47,0.027485,0.121228,0.001704,0.000213,GEN11_SP_N_92_ByTurnover,['Fluctuating']
80,['Irregular'],3851,0.276255,3740,111,0.028824,0.072307,0.001396,0.000175,GEN11_SP_N_92_ByTurnover,['Irregular']
81,['Other'],7282,0.522382,7048,234,0.032134,-0.039825,0.000844,0.000106,GEN11_SP_N_92_ByTurnover,['Other']
82,['Stable'],1097,0.078694,1058,39,0.035552,-0.144429,0.001758,0.000220,GEN11_SP_N_92_ByTurnover,['Stable']


In [309]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    '[', ''
)
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    ']', ''
)

In [310]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = '{' + df_intervals_mod[df_intervals_mod.Feature == col_name].Bin + '}'

In [311]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
79,{'Fluctuating'},1710,0.122669,1663,47,0.027485,0.121228,0.001704,0.000213,GEN11_SP_N_92_ByTurnover,['Fluctuating']
80,{'Irregular'},3851,0.276255,3740,111,0.028824,0.072307,0.001396,0.000175,GEN11_SP_N_92_ByTurnover,['Irregular']
81,{'Other'},7282,0.522382,7048,234,0.032134,-0.039825,0.000844,0.000106,GEN11_SP_N_92_ByTurnover,['Other']
82,{'Stable'},1097,0.078694,1058,39,0.035552,-0.144429,0.001758,0.000220,GEN11_SP_N_92_ByTurnover,['Stable']


### GEN11_SP_N_92_ByRP

In [312]:
col_name = 'GEN11_SP_N_92_ByRP'

In [313]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
83,['Other'],5719,0.410258,5550,169,0.029551,0.046651,0.000874,0.000109,GEN11_SP_N_92_ByRP,['Other']
84,"[SR, SNR]",7168,0.514204,6921,247,0.034459,-0.112076,0.006810,0.000851,GEN11_SP_N_92_ByRP,"[SR, SNR]"
85,[JNR],562,0.040316,552,10,0.017794,0.565960,0.009995,0.001233,GEN11_SP_N_92_ByRP,[JNR]
86,[JR],491,0.035222,486,5,0.010183,1.131767,0.027587,0.003275,GEN11_SP_N_92_ByRP,[JR]


In [314]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    '[', ''
)
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.str.replace(
    ']', ''
)

In [315]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = '{' + df_intervals_mod[df_intervals_mod.Feature == col_name].Bin + '}'

In [316]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
83,{'Other'},5719,0.410258,5550,169,0.029551,0.046651,0.000874,0.000109,GEN11_SP_N_92_ByRP,['Other']
84,"{SR, SNR}",7168,0.514204,6921,247,0.034459,-0.112076,0.006810,0.000851,GEN11_SP_N_92_ByRP,"[SR, SNR]"
85,{JNR},562,0.040316,552,10,0.017794,0.565960,0.009995,0.001233,GEN11_SP_N_92_ByRP,[JNR]
86,{JR},491,0.035222,486,5,0.010183,1.131767,0.027587,0.003275,GEN11_SP_N_92_ByRP,[JR]


### GEN11_SP_K_80_TO

In [317]:
col_name = 'GEN11_SP_K_80_TO'

In [318]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
87,['1234'],3218,0.230846,3172,46,0.014295,0.788473,0.100986,0.012306,GEN11_SP_K_80_TO,['1234']
88,['56'],740,0.053085,723,17,0.022973,0.305193,0.004296,0.000535,GEN11_SP_K_80_TO,['56']
89,['J' 'I'],2029,0.145552,1945,84,0.041400,-0.302803,0.015418,0.001920,GEN11_SP_K_80_TO,['J' 'I']
90,XX,2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190,GEN11_SP_K_80_TO,XX
91,T,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,GEN11_SP_K_80_TO,T
92,"[G, F, S, U]",5577,0.400072,5372,205,0.036758,-0.179058,0.013962,0.001743,GEN11_SP_K_80_TO,"[G, F, S, U]"


In [319]:
df_intervals_mod.loc[df_intervals_mod.Feature == col_name, 'Bin'] = df_intervals_mod[df_intervals_mod.Feature == col_name].Bin.replace(
    {
        "['1234']": "{1, 2, 3, 4}",
        "['56']": "{5, 6}",
        "['J' 'I']": "{'J', 'I'}",
        "XX": "{'XX'}",
        "T": "{'T'}",
        "[G, F, S, U]": "{'G', 'F', 'S', 'U'}",
    }
)


In [320]:
df_intervals_mod[df_intervals_mod.Feature == col_name]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
87,"{1, 2, 3, 4}",3218,0.230846,3172,46,0.014295,0.788473,0.100986,0.012306,GEN11_SP_K_80_TO,['1234']
88,"{5, 6}",740,0.053085,723,17,0.022973,0.305193,0.004296,0.000535,GEN11_SP_K_80_TO,['56']
89,"{'J', 'I'}",2029,0.145552,1945,84,0.041400,-0.302803,0.015418,0.001920,GEN11_SP_K_80_TO,['J' 'I']
90,{'XX'},2335,0.167504,2256,79,0.033833,-0.093103,0.001517,0.000190,GEN11_SP_K_80_TO,XX
91,{'T'},41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,GEN11_SP_K_80_TO,T
92,"{'G', 'F', 'S', 'U'}",5577,0.400072,5372,205,0.036758,-0.179058,0.013962,0.001743,GEN11_SP_K_80_TO,"[G, F, S, U]"


In [321]:
for feat in COLS_INTER:
    temp = df_intervals_mod[df_intervals_mod.Feature == feat][['Bin', 'Bin_init']]
    temp = temp[~temp.duplicated()].set_index('Bin_init')
    dict_to_repl = temp.to_dict(orient='dict')['Bin']
    # df[f'Bin{feat}'] = df[f'Bin{feat}'].replace(dict_to_repl)

In [322]:
df_intervals = pd.concat((df_intervals_mod, df_intervals_st)).reset_index(drop=True)

## Check

In [323]:
np.random.seed(RANDOM_SEED)
random_col = np.random.choice(list(COLUMNS_DICT.keys()), 1)[0]
df_intervals[df_intervals.Feature == random_col]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
14813,['T' 5.0 4.0 6.0],1733,0.124319,1710,23,0.013272,0.863751,0.063242,0.007668,AFF_T_64_01,NaN
14814,[3.0 1.0 2.0],1057,0.075825,1038,19,0.017975,0.555609,0.018198,0.002246,AFF_T_64_01,NaN
14815,['G'],4325,0.310258,4194,131,0.030289,0.021210,0.000138,0.000017,AFF_T_64_01,NaN
14816,['X' 'U'],2165,0.155308,2092,73,0.033718,-0.089587,0.001300,0.000162,AFF_T_64_01,NaN
14817,['F'],1324,0.094978,1275,49,0.037009,-0.186122,0.003594,0.000449,AFF_T_64_01,NaN
14818,['I' 'J'],2468,0.177044,2369,99,0.040113,-0.269900,0.014665,0.001828,AFF_T_64_01,NaN
14819,['S'],868,0.062267,831,37,0.042627,-0.333291,0.008110,0.001009,AFF_T_64_01,NaN


In [324]:
df_train[df_train[random_col] == -1].shape

(0, 1787)

In [325]:
df_train[df_train[random_col] == -1][TARGET_NAME].value_counts()

Series([], Name: count, dtype: int64)

In [326]:
df_train[df_train[random_col] == -1][TARGET_BIN_IisB].value_counts()

Series([], Name: count, dtype: int64)

In [327]:
df_intervals.to_csv('bins_optbins_neg_sep_I_excl.csv')

In [328]:
COLS_TO_SAVE = [
'OPTIN_11SCN_MLOptBin',
'G5_ND_SPECIIOptBin',
'SP_F2_33OptBin',
'AFF_B_09_01OptBin',
'TRD_A_20OptBin',
'TRD_A_08OptBin',
'VM01_SP_VM2_15OptBin',
# 'GEN11_SP_N_92OptBin',
'SP_G_37OptBin',
'E4_Q_17OptBin',
'TRD_C_07OptBin',
'TRD_B_20OptBin',
'SP_B2_18OptBin',
'SPA_F2_33OptBin',
'VM01_SP_VM2_14OptBin',
'VM01_SP_VM2_24OptBin',
'SP_G_38OptBin',
'TRD_O_05OptBin',
'E1_A_09OptBin',
'TRD_O_07OptBin',
'ND_INC_03OptBin',
'VM01_SP_VM2_17OptBin',
'SP_B1_14OptBin',
'GEN11_SP_N_91OptBin',
'GEN11_SP_EDI_07OptBin',
'ND_ECC_06OptBin',
'TRD_B_18OptBin',
'TRD_B_08OptBin',
'EA5_S_01OptBin',
'CLU_CLI_L6M_NPR_L6MOptBin',
'E1_B_06OptBin',
# 'GEN11_SP_K_80OptBin',
'SP_I_63_01OptBin',
'SP_B2_20OptBin',
'E4_Q_04OptBin',
'ND_ECC_04OptBin',
'VM08_SP_VM2_15OptBin',
'VAL_DELINQ_ACCSOptBin',
# 'ND_ERL_01OptBin',
'VM08_SP_VM2_20OptBin',
'EA1_D_02OptBin',
'AFF_E_07_01OptBin',
'TRD_O_06OptBin',
'E2_G_08OptBin',
'AFF_T_07_01OptBin',
'VM02_SP_VM1_18OptBin',
'GEN11_SP_K_80_TOOptBin',
'GEN11_SP_N_92_ByRPOptBin',
'GEN11_SP_N_92_ByTurnoverOptBin',
'GEN11_SP_N_92_IfSoleJointOtherAccOptBin',
'GEN11_SP_N_92_NotABOptBin',
'GEN11_SP_N_92_ABOtherOptBin'
]
df_intervals[df_intervals.Feature.isin([col[:-len('OptBin')] for col in COLS_TO_SAVE])].to_csv('bins_optbins_neg_sep_I_excl_sample.csv', index=None)

In [329]:
df_intervals

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature,Bin_init
0,"[1, inf)",465,0.033357,457,8,0.017204,0.000000,0.009164,0.001129,TRD_A_20,['1+']
1,"{-3, 0}",5977,0.428766,5932,45,0.007529,1.436451,0.480789,0.055412,TRD_A_20,"{-3, 0}"
2,-2,7457,0.534935,7079,378,0.050691,-0.515010,0.181803,0.022478,TRD_A_20,-2
3,-1,41,0.002941,41,0,0.000000,0.000000,0.000000,0.000000,TRD_A_20,-1
4,"[-100, 0.00)",4722,0.338737,4672,50,0.010589,1.092316,0.251052,0.029909,TRD_B_24,"(-inf, 0.00)"
...,...,...,...,...,...,...,...,...,...,...,...
16089,['N'],13029,0.934648,12641,388,0.029780,0.038692,0.001374,0.000172,EA4_Q_06,NaN
16090,['Y'],911,0.065352,868,43,0.047201,-0.440012,0.015627,0.001938,EA4_Q_06,NaN
16091,"(-1.0, inf)",3330,0.238881,3179,151,0.045345,-0.397961,0.045775,0.005684,VM03_SP_VM2_26,NaN
16092,-2.0,10569,0.758178,10289,280,0.026493,0.159038,0.017810,0.002224,VM03_SP_VM2_26,NaN


# Compute IV

In [330]:
df_iv = df_intervals.groupby('Feature').IV.sum().reset_index(name='IV')
df_iv.head()

,Feature,IV
0,AFF_B_09_01,0.969814
1,AFF_C_03_01,0.029781
2,AFF_C_06,0.049189
3,AFF_C_19_01,0.007170
4,AFF_E_07_01,0.274775


In [331]:
assert df_iv.shape[0] == len(COLUMNS_DICT)
assert not df_iv.IV.isna().any()

In [332]:
df_iv.to_csv('IV_optbins_neg_sep_I_excl.csv')

In [333]:
df.to_csv(os.path.join(DATA_PATH, 'final_aiq2_optbins_neg_sep_I_excl.csv'))